In [1]:
import numpy as np
import pandas as pd

In [2]:
# Function that returns number of rows and count of unique PatientIDs for a dataframe. 
def row_ID(dataframe):
    row = dataframe.shape[0]
    ID = dataframe['PatientID'].nunique()
    return row, ID

1. Load the full cohort previously defined

In [3]:
cohort = pd.read_csv('../checkpoint_trial/full_cohort.csv')

In [4]:
cohort.sample(3)

,PatientID,LineName,StartDate
2156,F34A7C73BDF17,Atezolizumab,2016-08-05
1684,F32A064EC1234,Pembrolizumab,2019-02-04
2868,F486864D8AF07,chemo,2020-03-23


In [5]:
cohort_IDs = cohort['PatientID'].to_numpy()

In [6]:
len(cohort_IDs)

6461

2. Load the demographics file and clean it

In [7]:
demographics = pd.read_csv('../data/Demographics.csv')

In [8]:
demographics.sample(3)

,PatientID,BirthYear,Gender,Race,Ethnicity,State
3974,F255CA8156F07,1955,M,Other Race,Hispanic or Latino,AZ
12214,FEC7D440CEFC6,1961,M,White,Not Hispanic or Latino,TN
4119,F7C49B6AF7D53,1942,M,White,Not Hispanic or Latino,AZ


In [9]:
#Filter for the patients in the cohort
demographics = demographics[demographics['PatientID'].isin(cohort_IDs)]

In [10]:
row_ID(demographics)

(6461, 6461)

In [11]:
demographics.sample(3)

,PatientID,BirthYear,Gender,Race,Ethnicity,State
10116,F2D321832276F,1947,M,White,Not Hispanic or Latino,NY
6231,F289050D8491F,1940,M,NaN,NaN,FL
8120,FF9CDCAF46F97,1937,F,White,Not Hispanic or Latino,IN


Race:
The recommendation from Flatiron is to do the following:
This approach specifically addresses the nuance of “Hispanic or Latino” appearing as both a Race and Ethnicity value in Flatiron data, as detailed in the Race and Ethnicity Overview. In order to align with OMB Standards, Flatiron recommends treating “Hispanic or Latino” as an ethnicity, using the following logic:

-Identify patients with a Race value of “Hispanic or Latino”
-For these patients, recode Race to NULL and Ethnicity to “Hispanic or Latino”

The resulting dataset will remove all instances of “Hispanic or Latino” as a Race, leaving “White,” “Black or African American,” “Asian,” “Other Race,” and NULL as possible Race values. 

In [12]:
demographics['Race'].value_counts()

Race
White                        4561
Other Race                    822
Black or African American     306
Asian                          86
Hispanic or Latino              8
Name: count, dtype: int64

In [13]:
# If race value is 'Hispanic or Latino', code as NULL, otherwise value unchanged.
demographics['race'] = (
    np.where(demographics['Race'] == 'Hispanic or Latino', 'NULL', demographics['Race'])
)

In [14]:
# Missing race value will be recoded as NULL
demographics['race'] = demographics['race'].fillna('NULL')

In [15]:
demographics['race'].value_counts().sum()

np.int64(6461)

In [16]:
demographics['race'].value_counts()

race
White                        4561
Other Race                    822
NULL                          686
Black or African American     306
Asian                          86
Name: count, dtype: int64

Ethnicity:

In [17]:
demographics['Ethnicity'].value_counts()

Ethnicity
Not Hispanic or Latino    4896
Hispanic or Latino         246
Name: count, dtype: int64

In [18]:
# If race value is equal to 'Hispanic or Latino', code ethnicity as 'Hispanic or Latino', otherwise unchanged. 
demographics['ethnicity'] = (
    np.where(demographics['Race'] == 'Hispanic or Latino', 'hispanic_latino', demographics['Ethnicity'])
)

In [19]:
#recode into snake case
demographics['ethnicity'] = demographics['ethnicity'].replace({'Hispanic or Latino': 'hispanic_latino'})

In [20]:
#recode into snake case
demographics['ethnicity'] = demographics['ethnicity'].replace({'Not Hispanic or Latino': 'not_hispanic_latino'})

In [21]:
demographics['ethnicity'].value_counts()

ethnicity
not_hispanic_latino    4896
hispanic_latino         247
Name: count, dtype: int64

In [22]:
#delete race and ethnicity columns
demographics = demographics.drop(columns = ['Race', 'Ethnicity'])

In [23]:
demographics.sample(3)

,PatientID,BirthYear,Gender,State,race,ethnicity
527,F7085E0551C9B,1936,F,NaN,White,not_hispanic_latino
4934,F2B7C57F87071,1938,M,CA,White,not_hispanic_latino
3790,F99BD0958C374,1938,F,AR,NULL,NaN


Per Flatiron, it is recommended that race and ethnicity are combined into a single variable as follows: 
-Hispanic or Latino 
-Not Hispanic or Latino, White 
-Not Hispanic or Latino, Black or African American 
-Not Hispanic or Latino, Asian 
-Not Hispanic or Latino, Other Race 
-Not Hispanic or Latino, Unknown Race 
-Unknown  
But will defer creating this column, given that there is some complexity in deciding how to handle cases where either race or ethnicity is unknown, will defer for now, given that race is unlikely to be central to the question at hand.

Birthyear, convert into Age; use the date of first line start (index date for this study) to calculate age

In [24]:
enhanced_adv = pd.read_csv('../data/Enhanced_AdvUrothelial.csv')

In [25]:
demographics = pd.merge(demographics, cohort[['PatientID', 'StartDate']], on = 'PatientID')

In [26]:
demographics.sample(3)

,PatientID,BirthYear,Gender,State,race,ethnicity,StartDate
3054,FA664F7997A10,1942,M,FL,Other Race,not_hispanic_latino,2018-03-05
2086,FB6B7369DD1B2,1937,M,CA,White,not_hispanic_latino,2012-09-11
1028,F4B0C44A47EFB,1969,M,NaN,White,not_hispanic_latino,2022-02-02


In [27]:
#assess data type for StartDate (is it a date)
print(demographics['StartDate'].dtype)

object


In [28]:
#how many entries are missing values
print(demographics['StartDate'].isnull().sum())

0


In [29]:
#what unique types of data are in the StartDate
print(demographics['StartDate'].apply(type).unique())

[<class 'str'>]


In [30]:
#convert StartDate to a date variable
demographics['StartDate'] = pd.to_datetime(demographics['StartDate'], format="%Y-%m-%d")

In [31]:
print(demographics['StartDate'].dtype)

datetime64[ns]


In [32]:
demographics.loc[:, 'age'] = demographics['StartDate'].dt.year - demographics['BirthYear']

In [33]:
demographics.sample(3)

,PatientID,BirthYear,Gender,State,race,ethnicity,StartDate,age
4770,F5C8F8DDAAE43,1933,F,NY,White,not_hispanic_latino,2018-04-03,85
341,FA60FB1776C75,1945,M,NaN,White,not_hispanic_latino,2022-06-21,77
3459,F12600412495E,1958,M,FL,NULL,NaN,2022-04-29,64


In [34]:
demographics = demographics.drop(columns = ['BirthYear', 'StartDate'])

In [35]:
demographics.sample(3)

,PatientID,Gender,State,race,ethnicity,age
4735,FB935846A6CE5,F,NM,White,not_hispanic_latino,74
5325,F026B6ABA5239,M,PA,White,not_hispanic_latino,66
2388,FB428DD0AB0A4,M,FL,Other Race,not_hispanic_latino,73


Practice type

In [36]:
practice = pd.read_csv('../data/Practice.csv')

In [37]:
#filter for patients in the cohort
practice = practice[practice['PatientID'].isin(cohort_IDs)]

In [38]:
row_ID(practice)

(7036, 6461)

In [39]:
practice_counts = practice['PracticeType'].value_counts()
print(practice_counts)

PracticeType
COMMUNITY    5751
ACADEMIC     1285
Name: count, dtype: int64


In cases where patients have multiple responses for PracticeType, need to address this by labeling as "BOTH"

In [40]:
#First determine how many practice types are present for each patient
practice_unique_count = (
    practice.groupby('PatientID')['PracticeType'].agg('nunique')
    .to_frame()
    .reset_index()
    .rename(columns = {'PracticeType': 'n_type'})
)

In [41]:
practice_n = pd.merge(practice, practice_unique_count, on = 'PatientID')

In [42]:
#Label patients with more than practice type with "BOTH"
practice_n['practice_type'] = (
    np.where(practice_n['n_type'] == 1, practice_n['PracticeType'], 'BOTH')
)

In [43]:
practice_n = (
    practice_n.drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'practice_type'])
)

In [44]:
row_ID(practice_n)

(6461, 6461)

In [45]:
practice_n.sample(3)

,PatientID,practice_type
5448,F13C6E150146E,ACADEMIC
972,F88B0CFA4F9EB,COMMUNITY
1283,F83296FBA633D,COMMUNITY


In [46]:
practice_n['practice_type'].value_counts()

practice_type
COMMUNITY    5179
ACADEMIC      816
BOTH          466
Name: count, dtype: int64

In [47]:
demographics = pd.merge(demographics, practice_n, on = 'PatientID')

In [48]:
demographics.sample(3)

,PatientID,Gender,State,race,ethnicity,age,practice_type
345,F0134DED49541,M,NaN,Black or African American,not_hispanic_latino,65,BOTH
2630,F63A1271E736D,M,FL,White,not_hispanic_latino,64,COMMUNITY
6146,F38760AE1DDD7,M,VA,White,not_hispanic_latino,77,COMMUNITY


Gender:

In [49]:
demographics['Gender'].value_counts()

Gender
M    4726
F    1733
Name: count, dtype: int64

Missing values noted, but will avoid imputation for now, with plans to address missingness in ultimate step; instead label as 'NULL'

In [52]:
demographics['Gender'].value_counts()

KeyError: 'Gender'

In [51]:
#Convert column name to snake
demographics = demographics.rename(columns = {'Gender': 'gender'})

In [53]:
demographics.sample(3)

,PatientID,gender,State,race,ethnicity,age,practice_type
5589,F762943D5E2E3,M,TN,Asian,not_hispanic_latino,76,COMMUNITY
3786,FE9C54481BD83,M,IL,Asian,not_hispanic_latino,83,COMMUNITY
3099,FF1FB78C306BC,F,FL,Other Race,NaN,85,COMMUNITY


In [54]:
demographics['gender'].value_counts()

gender
M       4726
F       1733
NULL       2
Name: count, dtype: int64

State:

In [55]:
# Group states into Census-Bureau regions  
state_dict = { 
    'ME': 'northeast', 
    'NH': 'northeast',
    'VT': 'northeast', 
    'MA': 'northeast',
    'CT': 'northeast',
    'RI': 'northeast',  
    'NY': 'northeast', 
    'NJ': 'northeast', 
    'PA': 'northeast', 
    'IL': 'midwest', 
    'IN': 'midwest', 
    'MI': 'midwest', 
    'OH': 'midwest', 
    'WI': 'midwest',
    'IA': 'midwest',
    'KS': 'midwest',
    'MN': 'midwest',
    'MO': 'midwest', 
    'NE': 'midwest',
    'ND': 'midwest',
    'SD': 'midwest',
    'DE': 'south',
    'FL': 'south',
    'GA': 'south',
    'MD': 'south',
    'NC': 'south', 
    'SC': 'south',
    'VA': 'south',
    'DC': 'south',
    'WV': 'south',
    'AL': 'south',
    'KY': 'south',
    'MS': 'south',
    'TN': 'south',
    'AR': 'south',
    'LA': 'south',
    'OK': 'south',
    'TX': 'south',
    'AZ': 'west',
    'CO': 'west',
    'ID': 'west',
    'MT': 'west',
    'NV': 'west',
    'NM': 'west',
    'UT': 'west',
    'WY': 'west',
    'AK': 'west',
    'CA': 'west',
    'HI': 'west',
    'OR': 'west',
    'WA': 'west',
    'PR': 'unknown'
}

demographics['region'] = demographics['State'].map(state_dict)

In [56]:
demographics['region'] = demographics['region'].fillna('unknown')

In [57]:
demographics['region'].value_counts()

region
south        2578
unknown      1523
west          809
northeast     799
midwest       752
Name: count, dtype: int64

In [58]:
demographics = demographics.drop(columns = ['State'])

In [59]:
demographics.sample(3)

,PatientID,gender,race,ethnicity,age,practice_type,region
3332,FC6EB361D9942,M,NULL,NaN,73,COMMUNITY,south
3863,F16FCE110172D,M,White,not_hispanic_latino,70,COMMUNITY,midwest
5891,F0DD1A3E6203A,M,White,not_hispanic_latino,74,COMMUNITY,south


In [60]:
%whos DataFrame

Variable                Type         Data/Info
----------------------------------------------
cohort                  DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics            DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
enhanced_adv            DataFrame               PatientID Diag<...>[13129 rows x 13 columns]
practice                DataFrame               PatientID     <...>\n[7036 rows x 4 columns]
practice_n              DataFrame              PatientID pract<...>\n[6461 rows x 2 columns]
practice_unique_count   DataFrame              PatientID  n_ty<...>\n[6461 rows x 2 columns]


In [61]:
#Keep cohort, demographics, enhanced_adv
del practice
del practice_n
del practice_unique_count

3. Clean enhanced_adv dataset

In [62]:
row_ID(enhanced_adv)

(13129, 13129)

In [63]:
#filter for patients in the cohort
enhanced_adv = enhanced_adv[enhanced_adv['PatientID'].isin(cohort_IDs)]

In [64]:
row_ID(enhanced_adv)

(6461, 6461)

In [65]:
enhanced_adv.sample(3)

,PatientID,DiagnosisDate,AdvancedDiagnosisDate,PrimarySite,DiseaseGrade,GroupStage,TStage,NStage,MStage,SmokingStatus,Surgery,SurgeryDate,SurgeryType
4249,F1CA5DEF2E54D,2017-01-01,2017-01-01,Ureter,High grade (G2/G3/G4),Stage IV,T3,N1,M0,History of smoking,True,2017-04-26,Nephroureterectomy
12699,F980D495AEFA9,2021-07-13,2021-07-13,Bladder,High grade (G2/G3/G4),Stage IVA,T4b,N0,M0,History of smoking,False,NaN,NaN
6419,FEA64BB4ACEA8,2006-12-20,2021-01-07,Bladder,High grade (G2/G3/G4),Unknown/not documented,Ta,Unknown/not documented,M0,History of smoking,False,NaN,NaN


GroupStage

In [66]:
stage_counts = enhanced_adv['GroupStage'].value_counts()
print(stage_counts)

GroupStage
Unknown/not documented    2978
Stage IV                  2055
Stage II                   459
Stage III                  263
Stage IVB                  172
Stage IIIA                 164
Stage IVA                  119
Stage I                    113
Stage IIIB                 104
Stage 0is                   20
Stage 0a                    14
Name: count, dtype: int64


In [67]:
# Dictionary for regrouping stages
stage_dict = { 
    'Stage 0': '0',
    'Stage 0is': '0',
    'Stage 0a': '0',
    'Stage I': 'I',
    'Stage II': 'II',
    'Stage III': 'III',
    'Stage IIIA': 'III',
    'Stage IIIB': 'III',
    'Stage IV': 'IV',
    'Stage IVA': 'IV',
    'Stage IVB': 'IV',
    'Unknown/not documented': 'unknown'
}

In [68]:
enhanced_adv['stage'] = enhanced_adv['GroupStage'].map(stage_dict)

In [69]:
stage_counts = enhanced_adv['stage'].value_counts()
print(stage_counts)

stage
unknown    2978
IV         2346
III         531
II          459
I           113
0            34
Name: count, dtype: int64


In [70]:
enhanced_adv = enhanced_adv.drop(columns = ['GroupStage'])

AdvancedDiagnosisDate

In [71]:
#make variable snake case
enhanced_adv = enhanced_adv.rename(columns = {'AdvancedDiagnosisDate': 'adv_diagnosis_date'})

In [72]:
#convert to date
enhanced_adv['adv_diagnosis_date'] = pd.to_datetime(enhanced_adv['adv_diagnosis_date'], format="%Y-%m-%d")

In [73]:
#confirm datetime conversion successful
print(enhanced_adv['adv_diagnosis_date'].dtype)

datetime64[ns]


In [74]:
enhanced_adv.loc[:, 'adv_diagnosis_year'] = enhanced_adv['adv_diagnosis_date'].dt.year

In [75]:
enhanced_adv.sample(3)

,PatientID,DiagnosisDate,adv_diagnosis_date,PrimarySite,DiseaseGrade,TStage,NStage,MStage,SmokingStatus,Surgery,SurgeryDate,SurgeryType,stage,adv_diagnosis_year
10404,F7A2B9AC82D68,2016-12-22,2016-12-22,Ureter,High grade (G2/G3/G4),T4,NX,M1,History of smoking,False,NaN,NaN,IV,2016
11128,F4BBEBD23749B,2019-01-25,2022-08-09,Bladder,High grade (G2/G3/G4),T2a,N0,M0,No history of smoking,False,NaN,NaN,II,2022
1222,F1A4256B6D54B,2016-02-24,2022-07-31,Bladder,High grade (G2/G3/G4),T1,Unknown/not documented,M0,No history of smoking,False,NaN,NaN,unknown,2022


DiagnosisDate

In [76]:
enhanced_adv = enhanced_adv.rename(columns = {'DiagnosisDate': 'diagnosis_date'})

In [77]:
enhanced_adv['diagnosis_date'] = pd.to_datetime(enhanced_adv['diagnosis_date'], format="%Y-%m-%d")

In [78]:
# Missing diagnosis_date will be replaced with adv_date; other dates will be left untouched. 
enhanced_adv['diagnosis_date'] = (
    np.where(enhanced_adv['diagnosis_date'].isna(), enhanced_adv['adv_diagnosis_date'], enhanced_adv['diagnosis_date'])
)

In [79]:
#confirm datetime conversion successful
print(enhanced_adv['adv_diagnosis_date'].dtype)

datetime64[ns]


Time from diagnosis date to advanced date

In [80]:
enhanced_adv.loc[:, 'delta_adv_diagnosis'] = (enhanced_adv['adv_diagnosis_date'] - enhanced_adv['diagnosis_date']).dt.days

In [81]:
enhanced_adv.sample(3)

,PatientID,diagnosis_date,adv_diagnosis_date,PrimarySite,DiseaseGrade,TStage,NStage,MStage,SmokingStatus,Surgery,SurgeryDate,SurgeryType,stage,adv_diagnosis_year,delta_adv_diagnosis
935,FB13861EC5370,2021-12-02,2021-12-02,Bladder,High grade (G2/G3/G4),T2,N1,M1,History of smoking,False,NaN,NaN,IV,2021,0
7898,F2EDE1E9A51BC,2012-03-01,2012-03-01,Bladder,High grade (G2/G3/G4),T4a,N2,Unknown/not documented,History of smoking,True,2012-05-08,Cystoprostatectomy,IV,2012,0
9654,F7399DE2CFEB9,2011-03-02,2011-03-02,Bladder,High grade (G2/G3/G4),T2a,N1,M0,No history of smoking,True,2011-08-05,Complete (radical) cystectomy,IV,2011,0


PrimarySite

In [82]:
enhanced_adv['PrimarySite'].value_counts()

PrimarySite
Bladder         4919
Renal Pelvis     897
Ureter           599
Urethra           46
Name: count, dtype: int64

In [83]:
enhanced_adv = enhanced_adv.rename(columns = {'PrimarySite': 'primary_site'})

In [84]:
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Bladder': 'lower_tract'})
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Urethra': 'lower_tract'})
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Renal Pelvis': 'upper_tract'})
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Ureter': 'upper_tract'})
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].fillna('unknown')

In [85]:
site_counts = enhanced_adv['primary_site'].value_counts()
print(site_counts)

primary_site
lower_tract    4965
upper_tract    1496
Name: count, dtype: int64


DiseaseGrade

In [86]:
enhanced_adv['DiseaseGrade'].value_counts()

DiseaseGrade
High grade (G2/G3/G4)     5441
Unknown/not documented     711
Low grade (G1)             309
Name: count, dtype: int64

In [87]:
enhanced_adv = enhanced_adv.rename(columns = {'DiseaseGrade': 'disease_grade'})

In [88]:
enhanced_adv['disease_grade'] = enhanced_adv['disease_grade'].replace({'High grade (G2/G3/G4)': 'high_grade'})
enhanced_adv['disease_grade'] = enhanced_adv['disease_grade'].replace({'Unknown/not documented': 'unknown'})
enhanced_adv['disease_grade'] = enhanced_adv['disease_grade'].replace({'Low grade (G1)': 'low_grade'})

In [89]:
enhanced_adv['disease_grade'].value_counts()

disease_grade
high_grade    5441
unknown        711
low_grade      309
Name: count, dtype: int64

TStage

In [90]:
enhanced_adv['TStage'].value_counts()

TStage
Unknown/not documented    2226
T2                        1140
T3                         738
T1                         720
T4                         295
T4a                        232
T3a                        200
T3b                        196
T2b                        192
T2a                        158
TX                         136
Ta                         126
T4b                         54
Tis                         42
T0                           6
Name: count, dtype: int64

In [91]:
enhanced_adv = enhanced_adv.rename(columns = {'TStage': 't_stage'})

In [92]:
# Dictionary for regrouping t stages
t_stage_dict = { 
    'Unknown/not documented': 'unknown',
    'T2': 'T2',
    'T3': 'T3',
    'T1': 'T1',
    'T4': 'T4',
    'T4a': 'T4',
    'T3a': 'T3',
    'T3b': 'T3',
    'T2b': 'T2',
    'T2a': 'T2',
    'TX': 'unknown',
    'Ta': 'Ta',
    'T4b': 'T4',
    'Tis': 'Tis',
    'T0': 'T0'
    
}

enhanced_adv['t_stage'] = enhanced_adv['t_stage'].map(t_stage_dict)

In [93]:
enhanced_adv['t_stage'].value_counts()

t_stage
unknown    2362
T2         1490
T3         1134
T1          720
T4          581
Ta          126
Tis          42
T0            6
Name: count, dtype: int64

NStage

In [94]:
enhanced_adv['NStage'].value_counts()

NStage
Unknown/not documented    3107
N0                        1315
NX                         743
N2                         612
N1                         497
N3                         187
Name: count, dtype: int64

In [95]:
enhanced_adv = enhanced_adv.rename(columns = {'NStage': 'n_stage'})

In [96]:
# Dictionary for regrouping n stages
n_stage_dict = { 
    'Unknown/not documented': 'unknown',
    'N0': 'N0',
    'NX': 'unknown',
    'N2': 'N2',
    'N1': 'N1',
    'N3': 'N3'
}

enhanced_adv['n_stage'] = enhanced_adv['n_stage'].map(n_stage_dict)

In [97]:
enhanced_adv['n_stage'].value_counts()

n_stage
unknown    3850
N0         1315
N2          612
N1          497
N3          187
Name: count, dtype: int64

MStage

In [98]:
enhanced_adv['MStage'].value_counts()

MStage
M0                        2630
Unknown/not documented    1878
M1                        1330
MX                         393
M1b                        139
M1a                         91
Name: count, dtype: int64

In [99]:
enhanced_adv = enhanced_adv.rename(columns = {'MStage': 'm_stage'})

In [100]:
# Dictionary for regrouping m stages
m_stage_dict = { 
    'M0': 'M0',
    'Unknown/not documented': 'unknown',
    'M1': 'M1',
    'MX': 'unknown',
    'M1b': 'M1',
    'M1a': 'M1'
}

enhanced_adv['m_stage'] = enhanced_adv['m_stage'].map(m_stage_dict)

In [101]:
enhanced_adv['m_stage'].value_counts()

m_stage
M0         2630
unknown    2271
M1         1560
Name: count, dtype: int64

SmokingStatus

In [102]:
enhanced_adv['SmokingStatus'].value_counts()

SmokingStatus
History of smoking        4742
No history of smoking     1677
Unknown/not documented      42
Name: count, dtype: int64

In [103]:
enhanced_adv = enhanced_adv.rename(columns = {'SmokingStatus': 'smoking_status'})

In [104]:
enhanced_adv['smoking_status'] = enhanced_adv['smoking_status'].replace({'History of smoking': 'smoker'})
enhanced_adv['smoking_status'] = enhanced_adv['smoking_status'].replace({'No history of smoking': 'never_smoker'})
enhanced_adv['smoking_status'] = enhanced_adv['smoking_status'].replace({'Unknown/not documented': 'unknown'})

In [105]:
enhanced_adv['smoking_status'].value_counts()

smoking_status
smoker          4742
never_smoker    1677
unknown           42
Name: count, dtype: int64

Surgery

In [106]:
enhanced_adv['Surgery'].value_counts()

Surgery
False    3345
True     3116
Name: count, dtype: int64

In [107]:
enhanced_adv = enhanced_adv.rename(columns = {'Surgery': 'surgery_status'})

In [108]:
enhanced_adv['surgery_status'].value_counts()

surgery_status
False    3345
True     3116
Name: count, dtype: int64

In [109]:
print(enhanced_adv['surgery_status'].dtype)

bool


SurgeryDate

In [110]:
print(enhanced_adv['SurgeryDate'].dtype)

object


In [111]:
enhanced_adv = enhanced_adv.rename(columns = {'SurgeryDate': 'surgery_date'})

In [112]:
enhanced_adv['surgery_date'] = pd.to_datetime(enhanced_adv['surgery_date'], format="%Y-%m-%d")

In [113]:
print(enhanced_adv['surgery_date'].dtype)

datetime64[ns]


Leaving cases where there is no surgery date, empty for now, preserving date formatting

SurgeryType

In [114]:
enhanced_adv['SurgeryType'].value_counts()

SurgeryType
Cystoprostatectomy               1210
Nephroureterectomy                882
Complete (radical) cystectomy     539
Partial cystectomy                126
Nephrectomy                       124
Ureterectomy                       98
Cystectomy, NOS                    73
Other                              58
Unknown/not documented              4
Urethrectomy                        2
Name: count, dtype: int64

In [115]:
enhanced_adv = enhanced_adv.rename(columns = {'SurgeryType': 'surgery_type'})

In [116]:
# Dictionary for regrouping surgery type; may need to consider alternative strategy of cystectomy vs. other surgery?
surgery_type_dict = { 
    'Cystoprostatectomy': 'lower_tract_resection',
    'Nephroureterectomy': 'upper_tract_resection',
    'Complete (radical) cystectomy': 'lower_tract_resection',
    'Partial cystectomy': 'lower_tract_resection',
    'Nephrectomy': 'upper_tract_resection',
    'Ureterectomy': 'upper_tract_resection',
    'Cystectomy, NOS': 'lower_tract_resection',
    'Other': 'unknown_surgery',
    'Unknown/not documented': 'unknown_surgery',
    'Urethrectomy': 'lower_tract_resection'
}

enhanced_adv['surgery_type'] = enhanced_adv['surgery_type'].map(surgery_type_dict)

In [117]:
enhanced_adv['surgery_type'] = enhanced_adv['surgery_type'].fillna('no_surgery')

In [118]:
enhanced_adv['surgery_type'].value_counts()

surgery_type
no_surgery               3345
lower_tract_resection    1950
upper_tract_resection    1104
unknown_surgery            62
Name: count, dtype: int64

In [120]:
#Now calculate times from diagnosis to therapy and from surgery to therapy initiation
enhanced_adv = pd.merge(enhanced_adv, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')
enhanced_adv['StartDate'] = pd.to_datetime(enhanced_adv['StartDate'], format="%Y-%m-%d")

In [121]:
enhanced_adv.loc[:, 'delta_therapy_surgery'] = (enhanced_adv['StartDate'] - enhanced_adv['surgery_date']).dt.days
enhanced_adv.loc[:, 'delta_therapy_diagnosis'] = (enhanced_adv['StartDate'] - enhanced_adv['diagnosis_date']).dt.days
enhanced_adv.sample(3)

,PatientID,diagnosis_date,adv_diagnosis_date,primary_site,disease_grade,t_stage,n_stage,m_stage,smoking_status,surgery_status,surgery_date,surgery_type,stage,adv_diagnosis_year,delta_adv_diagnosis,StartDate,delta_therapy_surgery,delta_therapy_diagnosis
3969,FE621D9BB6490,2017-12-18,2017-12-18,lower_tract,high_grade,T4,N3,M0,smoker,False,NaT,no_surgery,IV,2017,0,2018-08-24,NaN,249
4139,F4A0A8294DA46,2020-06-03,2020-06-03,lower_tract,high_grade,T4,N1,M0,smoker,True,2020-07-15,lower_tract_resection,III,2020,0,2020-12-29,167.0,209
2105,F8DE63D1C2C1E,2016-04-05,2020-06-18,lower_tract,high_grade,unknown,unknown,M0,never_smoker,False,NaT,no_surgery,unknown,2020,1535,2020-07-06,NaN,1553


In [122]:
enhanced_adv = enhanced_adv.drop(columns = ['surgery_date', 'StartDate', 'diagnosis_date', 'adv_diagnosis_date'])
#Final enhanced_adv dataframe
enhanced_adv.sample(3)

,PatientID,diagnosis_date,adv_diagnosis_date,primary_site,disease_grade,t_stage,n_stage,m_stage,smoking_status,surgery_status,surgery_type,stage,adv_diagnosis_year,delta_adv_diagnosis,delta_therapy_surgery,delta_therapy_diagnosis
5299,F006AD96597CA,2017-06-27,2017-11-14,lower_tract,high_grade,unknown,unknown,unknown,smoker,True,lower_tract_resection,unknown,2017,140,-20.0,211
6270,F585AEC452010,2012-03-30,2013-09-06,lower_tract,high_grade,T4,unknown,M0,smoker,True,lower_tract_resection,unknown,2013,525,395.0,545
1179,FF7E5499591E4,2013-09-25,2013-09-25,lower_tract,high_grade,T4,N2,M0,smoker,True,lower_tract_resection,III,2013,0,112.0,161


4. Clean Biomarkers dataset 

In [123]:
biomarkers = pd.read_csv('../data/Enhanced_AdvUrothelialBiomarkers.csv')

In [124]:
biomarkers = biomarkers[biomarkers['PatientID'].isin(cohort_IDs)]

In [125]:
biomarkers.sample(3)

,PatientID,BiomarkerName,CellType,SpecimenCollectedDate,SpecimenReceivedDate,ResultDate,BiomarkerStatus,Gene,BiomarkerDetail,ExpressionLevel,SampleType,TissueCollectionSite,TestType,LabName,Assay,IHCClone,StainingIntensity,PercentStaining,CombinedPositiveScore
8427,FD738DE4975C9,FGFR,NaN,2017-02-14,2017-03-01,2017-03-10,Negative,NaN,NaN,NaN,Tissue,Metastatic site,NGS,Foundation Medicine,NaN,NaN,NaN,NaN,NaN
8681,F3C225BF5F32F,PDL1,Tumor cells,2019-08-22,2019-11-01,2019-11-01,No interpretation given in report,NaN,NaN,NaN,Tissue,Primary site,IHC,In-house pathology lab,Dako PD-L1 IHC 22C3 pharmDx (Keytruda companio...,22C3,NaN,< 1%,NaN
3920,FBFF1A0F970BA,PDL1,Immune cells,2021-03-03,2021-04-20,2021-05-03,PD-L1 negative/not detected,NaN,NaN,NaN,Tissue,Primary site,IHC,Caris Life Sciences,Ventana PD-L1 (SP142) Assay (Tecentriq complem...,SP142,NaN,1%,NaN


In [126]:
biomarkers['BiomarkerName'].value_counts()

BiomarkerName
PDL1    3200
FGFR    3126
Name: count, dtype: int64

The biomarker file is in Long format. Thus the goal will be to transition to wide, per patient format.

Given that the only two biomarkers in the dataset are FGFR and PDL1, we will want to report the FGFR test result and the PDL1 test result. For FGFR, the result will be positive or negative. For PDL1, the result of interest will be more complicated (report of positive vs. negative, as well as potentially the CPS score).

The other important piece of data for biomarker reporting is the time point of the biomarker. We will want to include biomarker test prior to the start date, as these are this the biomarker info associated with the tumor at the time of the treatment initiation (that was known). We will include biomarkers that result within 30 days of the treatment start date.

Regarding biomarker date information, result date is the date the biomarker result was first reported, and so represents the date on which the clinician would be expected to have information about the patient’s biomarker status to inform the course of treatment. Flatiron recommends using result date as the relevant biomarker test date and using specimen received date as the proxy when result date is not available. The gaps between collected date and either received or result date are substantially more variable.

We'll begin by imputing specimen received date when result date is missing. Then, we'll select all biomarkers that fall within the elbility window.

In [127]:
#convert to date
biomarkers['ResultDate'] = pd.to_datetime(biomarkers['ResultDate'], format="%Y-%m-%d")

In [128]:
print(biomarkers['ResultDate'].dtype)

datetime64[ns]


In [129]:
#convert to date
biomarkers['SpecimenReceivedDate'] = pd.to_datetime(biomarkers['SpecimenReceivedDate'], format="%Y-%m-%d")
print(biomarkers['SpecimenReceivedDate'].dtype)

datetime64[ns]


In [130]:
# Replace missing result date with specimen received date. 
biomarkers.loc[:, 'result_date'] = (
    np.where(biomarkers['ResultDate'].isna(), biomarkers['SpecimenReceivedDate'], biomarkers['ResultDate'])
)

In [131]:
biomarkers.sample(3)

,PatientID,BiomarkerName,CellType,SpecimenCollectedDate,SpecimenReceivedDate,ResultDate,BiomarkerStatus,Gene,BiomarkerDetail,ExpressionLevel,SampleType,TissueCollectionSite,TestType,LabName,Assay,IHCClone,StainingIntensity,PercentStaining,CombinedPositiveScore,result_date
4829,F5475EB8E2F7A,FGFR,NaN,2021-12-06,2021-12-18,2021-12-30,Positive,FGFR4,Variant of Unknown Significance (VUS),NaN,Tissue,Metastatic site,NGS,Caris Life Sciences,NaN,NaN,NaN,NaN,NaN,2021-12-30
549,F603EE6BD2E4B,FGFR,NaN,2021-07-27,2021-08-20,2021-08-26,Unsuccessful/indeterminate test,NaN,NaN,NaN,Tissue,Metastatic site,NGS,Quest Diagnostics,NaN,NaN,NaN,NaN,NaN,2021-08-26
3468,F89E007058451,FGFR,NaN,2022-05-25,2022-08-09,2022-08-23,Negative,NaN,NaN,NaN,Tissue,Primary site,NGS,Caris Life Sciences,NaN,NaN,NaN,NaN,NaN,2022-08-23


In [132]:
biomarkers = pd.merge(biomarkers, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [133]:
biomarkers.sample(3)

,PatientID,BiomarkerName,CellType,SpecimenCollectedDate,SpecimenReceivedDate,ResultDate,BiomarkerStatus,Gene,BiomarkerDetail,ExpressionLevel,...,TissueCollectionSite,TestType,LabName,Assay,IHCClone,StainingIntensity,PercentStaining,CombinedPositiveScore,result_date,StartDate
1944,F1EDAD9D9582E,PDL1,Immune cells,2019-12-20,2020-04-08,2020-04-24,PD-L1 negative/not detected,NaN,NaN,NaN,...,Primary site,IHC,Caris Life Sciences,Unknown/not documented,SP142,NaN,0%,NaN,2020-04-24,2020-02-07
2699,FE3CF05CE50BB,FGFR,NaN,2020-02-20,2020-03-03,2020-03-06,Negative,NaN,NaN,NaN,...,Metastatic site,NGS,Paradigm,NaN,NaN,NaN,NaN,NaN,2020-03-06,2020-02-27
128,F62682F1B83E9,PDL1,Immune cells,2022-08-17,2022-09-02,2022-09-20,PD-L1 negative/not detected,NaN,NaN,NaN,...,Primary site,IHC,Caris Life Sciences,Unknown/not documented,SP142,NaN,0%,NaN,2022-09-20,2020-04-23


In [134]:
# Create new variable that captures difference in days between result date and start date. 
biomarkers['StartDate'] = pd.to_datetime(biomarkers['StartDate'], format="%Y-%m-%d")
biomarkers.loc[:, 'bio_date_diff'] = (biomarkers['result_date'] - biomarkers['StartDate']).dt.days

In [135]:
biomarkers.sample(3)

,PatientID,BiomarkerName,CellType,SpecimenCollectedDate,SpecimenReceivedDate,ResultDate,BiomarkerStatus,Gene,BiomarkerDetail,ExpressionLevel,...,TestType,LabName,Assay,IHCClone,StainingIntensity,PercentStaining,CombinedPositiveScore,result_date,StartDate,bio_date_diff
4162,F9C8AF68C73A6,FGFR,NaN,2023-06-01,2023-06-08,2023-06-29,Positive,FGFR4,Variant of Unknown Significance (VUS),NaN,...,NGS,Caris Life Sciences,NaN,NaN,NaN,NaN,NaN,2023-06-29,2023-06-22,7.0
1643,F4B1DBA612436,FGFR,NaN,2022-01-12,2022-01-13,2022-01-19,Negative,NaN,NaN,NaN,...,NGS,Guardant,NaN,NaN,NaN,NaN,NaN,2022-01-19,2022-01-24,-5.0
1812,F841C280F2633,PDL1,Immune cells,2021-11-04,2021-11-12,2021-11-29,PD-L1 negative/not detected,NaN,NaN,NaN,...,IHC,Caris Life Sciences,Lab-developed test,SP142,NaN,0%,NaN,2021-11-29,2021-11-16,13.0


In [136]:
# Select all patients with biomarkers < +30 from metastatic treatment start. 
biomarker_win = biomarkers[biomarkers['bio_date_diff'] <= 30]

In [137]:
row_ID(biomarker_win)

(3814, 1682)

In [138]:
biomarker_win['BiomarkerName'].value_counts()

BiomarkerName
PDL1    2157
FGFR    1657
Name: count, dtype: int64

In [139]:
biomarker_win['BiomarkerStatus'].value_counts()

BiomarkerStatus
Negative                             1167
No interpretation given in report    1045
PD-L1 negative/not detected           746
Positive                              409
PD-L1 positive                        293
Unsuccessful/indeterminate test       128
Results pending                        15
Unknown                                10
PD-L1 equivocal                         1
Name: count, dtype: int64

In [140]:
#Start with FGFR
biomarker_fgfr = (
    biomarker_win
    .query('BiomarkerName == "FGFR"')
)

In [141]:
row_ID(biomarker_fgfr)

(1657, 1162)

In [142]:
biomarker_fgfr['BiomarkerStatus'].value_counts()

BiomarkerStatus
Negative                           1167
Positive                            409
Unsuccessful/indeterminate test      78
Results pending                       3
Name: count, dtype: int64

In [143]:
#Refine indicator variable
fgfr_dict = { 
    'Positive': 'Positive',
    'Negative': 'Negative',
    'Unsuccessful/indeterminate test': 'Missing',
    'Results pending': 'Missing'
}

biomarker_fgfr.loc[:, 'bio_status'] = biomarker_fgfr['BiomarkerStatus'].map(fgfr_dict)

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/2627664016.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biomarker_fgfr.loc[:, 'bio_status'] = biomarker_fgfr['BiomarkerStatus'].map(fgfr_dict)


In [144]:
biomarker_fgfr.sample(3)

,PatientID,BiomarkerName,CellType,SpecimenCollectedDate,SpecimenReceivedDate,ResultDate,BiomarkerStatus,Gene,BiomarkerDetail,ExpressionLevel,...,LabName,Assay,IHCClone,StainingIntensity,PercentStaining,CombinedPositiveScore,result_date,StartDate,bio_date_diff,bio_status
1679,FCD30F264D87A,FGFR,NaN,2021-06-02,2022-11-15,2022-11-27,Negative,NaN,NaN,NaN,...,Caris Life Sciences,NaN,NaN,NaN,NaN,NaN,2022-11-27,2022-12-16,-19.0,Negative
3939,F43CE0F168B14,FGFR,NaN,2023-02-20,2023-02-24,2023-03-13,Negative,NaN,NaN,NaN,...,Caris Life Sciences,NaN,NaN,NaN,NaN,NaN,2023-03-13,2023-03-14,-1.0,Negative
3054,F8B7A9D24E573,FGFR,NaN,2022-08-30,2022-09-01,2022-09-09,Negative,NaN,NaN,NaN,...,Guardant,NaN,NaN,NaN,NaN,NaN,2022-09-09,2022-09-12,-3.0,Negative


In [145]:
biomarker_fgfr['bio_status'].value_counts()

bio_status
Negative    1167
Positive     409
Missing       81
Name: count, dtype: int64

In [146]:
#Now convert FGFR data to wide; note that reverse alphabetical order will work to prioritize positive result over negative over missing for the bio_status variable
biomarker_fgfr_wide = (
    biomarker_fgfr
    .sort_values(by = ['PatientID', 'BiomarkerName', 'result_date', 'bio_status'], ascending = [True, True, False, False])
    .drop_duplicates(subset = ['PatientID', 'BiomarkerName'], keep = 'first')
    .pivot(index = 'PatientID', columns = 'BiomarkerName', values = 'bio_status')
    .reset_index()
)

In [147]:
biomarker_fgfr_wide.sample(3)

BiomarkerName,PatientID,FGFR
1038,FE1737315C7BD,Negative
837,FB6E2826C73F8,Positive
860,FBBACB7A08086,Negative


In [148]:
biomarker_fgfr_wide['FGFR'].value_counts()

FGFR
Negative    790
Positive    329
Missing      43
Name: count, dtype: int64

In [149]:
row_ID(biomarker_fgfr_wide)

(1162, 1162)

In [150]:
biomarker_fgfr_wide = (
    pd.concat(
        [biomarker_fgfr_wide, 
         pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(biomarker_fgfr_wide['PatientID'])].to_frame(name='PatientID')],
        ignore_index=True,  # This ensures that the index is reset after concatenation
        sort=False
    )
    .fillna("Missing")
)

In [151]:
row_ID(biomarker_fgfr_wide)

(6461, 6461)

In [152]:
biomarker_fgfr_wide['FGFR'].value_counts()

FGFR
Missing     5342
Negative     790
Positive     329
Name: count, dtype: int64

Now turn to PD-L1. 
Per Flatiron, "Given the impact of trends in clinical practice on completeness over time, Flatiron recommends defaulting to PercentStaining as the primary source of truth to assess PD-L1 status over other options in the Biomarker table. Assessment of PercentStaining also enables the researcher to set relevant thresholds for a binary (e.g., “positive”, “negative”) or categorical (e.g., “≥1%”, “1-49%”, “50-100%”) PD-L1 patient-level status, depending on the research question."

Therefore, we will plan to just report percent staining.

In [153]:
biomarker_pdl1 = (
    biomarker_win
    .query('BiomarkerName == "PDL1"')
)

In [154]:
row_ID(biomarker_pdl1)

(2157, 1342)

In [155]:
biomarker_pdl1.sample(3)

,PatientID,BiomarkerName,CellType,SpecimenCollectedDate,SpecimenReceivedDate,ResultDate,BiomarkerStatus,Gene,BiomarkerDetail,ExpressionLevel,...,TestType,LabName,Assay,IHCClone,StainingIntensity,PercentStaining,CombinedPositiveScore,result_date,StartDate,bio_date_diff
3555,F2E939061C06B,PDL1,Combined tumor and immune cells (CPS),2023-09-01,2023-10-04,2023-10-14,PD-L1 negative/not detected,NaN,NaN,NaN,...,IHC,Caris Life Sciences,Dako PD-L1 IHC 22C3 pharmDx (Keytruda companio...,22C3,NaN,NaN,1,2023-10-14,2023-10-17,-3.0
962,F13C6E150146E,PDL1,Combined tumor and immune cells (CPS),2017-04-06,2018-10-19,2018-11-05,No interpretation given in report,NaN,NaN,NaN,...,IHC,Unknown,Unknown/not documented,22C3,NaN,NaN,Unknown/not documented,2018-11-05,2019-01-03,-59.0
74,FFEB6981A498B,PDL1,Unknown/not documented,NaN,2022-04-19,NaT,Unknown,NaN,NaN,NaN,...,Unknown,Guardant,Unknown/not documented,Unknown/not documented,NaN,NaN,NaN,2022-04-19,2022-04-26,-7.0


In [156]:
biomarker_pdl1['PercentStaining'].value_counts()

PercentStaining
0%           343
1%           117
< 1%         112
5% - 9%      105
2% - 4%      101
10% - 19%     80
20% - 29%     52
30% - 39%     33
90% - 99%     15
80% - 89%     12
50% - 59%      9
70% - 79%      9
60% - 69%      8
40% - 49%      8
100%           3
Name: count, dtype: int64

In [157]:
pdl1_dict = { 
    np.nan: 0,
    '0%': 1, 
    '< 1%': 2,
    '1%': 3, 
    '2% - 4%': 4,
    '5% - 9%': 5,
    '10% - 19%': 6,  
    '20% - 29%': 7, 
    '30% - 39%': 8, 
    '40% - 49%': 9, 
    '50% - 59%': 10, 
    '60% - 69%': 11, 
    '70% - 79%': 12, 
    '80% - 89%': 13, 
    '90% - 99%': 14,
    '100%': 15
}

biomarker_pdl1.loc[:, 'percent_staining'] = biomarker_pdl1['PercentStaining'].map(pdl1_dict)

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/193486197.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biomarker_pdl1.loc[:, 'percent_staining'] = biomarker_pdl1['PercentStaining'].map(pdl1_dict)


In [158]:
biomarker_pdl1['percent_staining'].value_counts()

percent_staining
0     1150
1      343
3      117
2      112
5      105
4      101
6       80
7       52
8       33
14      15
13      12
10       9
12       9
11       8
9        8
15       3
Name: count, dtype: int64

In [159]:
biomarker_pdl1_staining = (
    biomarker_pdl1
    .sort_values(by = ['PatientID', 'percent_staining'], ascending = False)
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .pivot(index = 'PatientID', columns = 'BiomarkerName', values = 'percent_staining')
    .rename(columns = {'PDL1': 'pdl1_staining'})
    .reset_index()
)
biomarker_pdl1_staining.columns.name = None

In [160]:
biomarker_pdl1_staining.sample(3)

,PatientID,pdl1_staining
1298,FF521340FE81E,1
591,F72BC04B5CDBE,0
1022,FC0AA4FC866F8,1


In [161]:
biomarker_pdl1_staining['pdl1_staining'].value_counts()

pdl1_staining
0     601
1     221
3      96
5      81
4      79
2      75
6      59
7      42
8      27
14     14
13     12
12      9
11      8
10      8
9       7
15      3
Name: count, dtype: int64

In [162]:
row_ID(biomarker_pdl1_staining)

(1342, 1342)

In [163]:
#For this study, will bucket PD-L1 into 0, 1-49, and 50-100% buckets
pdl1_dict_rev = { 
    0: np.nan,
    1: '0%', 
    2: '0%',
    3: '>=1%', 
    4: '>=1%',
    5: '>=1%',
    6: '>=1%',  
    7: '>=1%', 
    8: '>=1%', 
    9: '>=1%', 
    10: '>=1%', 
    11: '>=1%', 
    12: '>=1%', 
    13: '>=1%', 
    14: '>=1%',
    15: '>=1%'
}

biomarker_pdl1_staining.loc[:, 'pdl1_staining'] = biomarker_pdl1_staining['pdl1_staining'].map(pdl1_dict_rev)

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/1641147471.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['>=1%' nan nan ... '>=1%' '>=1%' '>=1%']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  biomarker_pdl1_staining.loc[:, 'pdl1_staining'] = biomarker_pdl1_staining['pdl1_staining'].map(pdl1_dict_rev)


In [164]:
biomarker_pdl1_staining['pdl1_staining'].value_counts()

pdl1_staining
>=1%    445
0%      296
Name: count, dtype: int64

In [165]:
biomarker_pdl1_staining.sample(3)

,PatientID,pdl1_staining
325,F3EF71BDBE709,0%
1027,FC14F0C9DC80B,0%
60,F0B64466B4294,>=1%


In [166]:
row_ID(biomarker_pdl1_staining)

(1342, 1342)

In [167]:
biomarker_pdl1_staining_wide = (
    pd.concat(
        [biomarker_pdl1_staining, 
         pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(biomarker_pdl1_staining['PatientID'])].to_frame(name='PatientID')],
        ignore_index=True,  # This ensures that the index is reset after concatenation
        sort=False
    )
    .fillna("Missing")
)

In [168]:
row_ID(biomarker_pdl1_staining_wide)

(6461, 6461)

In [169]:
biomarker_pdl1_staining_wide['pdl1_staining'].value_counts()

pdl1_staining
Missing    5720
>=1%        445
0%          296
Name: count, dtype: int64

Flatiron recommends considering using BiomarkerStatus to impute the missing patient-level PercentStaining category value. Impute missing PercentStaining values as follows:

-Impute a PercentStaining value of “≥1%” for patients with at least one confirmed positive PD-L1 result within the eligible window.
-Impute a PercentStaining value of “0%” to patients with no confirmed positive PD-L1 results and at least one confirmed negative PD-L1 result within the eligible window.
-Do not impute a PercentStaining value to patients who have no confirmed positive or negative PD-L1 results within the eligible window.

In [170]:
biomarker_pdl1.BiomarkerStatus.value_counts(dropna = False)

BiomarkerStatus
No interpretation given in report    1045
PD-L1 negative/not detected           746
PD-L1 positive                        293
Unsuccessful/indeterminate test        50
Results pending                        12
Unknown                                10
PD-L1 equivocal                         1
Name: count, dtype: int64

In [171]:
# Create indicator variable where where 2 if positive, 1 if negative, and 0 if unknown or missing. 
conditions = [
    (biomarker_pdl1['BiomarkerStatus'] == 'PD-L1 positive'),
    (biomarker_pdl1['BiomarkerStatus'] == 'PD-L1 negative/not detected')
]

choices = [2,1]
biomarker_pdl1.loc[:, 'bio_status'] = np.select(conditions, choices, default = 0)

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/3284090208.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biomarker_pdl1.loc[:, 'bio_status'] = np.select(conditions, choices, default = 0)


In [172]:
biomarker_pdl1.sample(3)

,PatientID,BiomarkerName,CellType,SpecimenCollectedDate,SpecimenReceivedDate,ResultDate,BiomarkerStatus,Gene,BiomarkerDetail,ExpressionLevel,...,Assay,IHCClone,StainingIntensity,PercentStaining,CombinedPositiveScore,result_date,StartDate,bio_date_diff,percent_staining,bio_status
522,F273EB6E559D0,PDL1,Combined tumor and immune cells (CPS),2021-02-09,2021-02-20,2021-02-23,PD-L1 negative/not detected,NaN,NaN,NaN,...,Dako PD-L1 IHC 22C3 pharmDx (Keytruda companio...,22C3,NaN,NaN,0,2021-02-23,2021-03-04,-9.0,0,1
1842,F0A3495E21509,PDL1,Combined tumor and immune cells (CPS),2020-09-02,2020-09-23,2020-09-30,PD-L1 negative/not detected,NaN,NaN,NaN,...,Dako PD-L1 IHC 22C3 pharmDx (Keytruda companio...,22C3,NaN,NaN,5-9,2020-09-30,2020-10-19,-19.0,0,1
488,F05B243AAED1E,PDL1,Combined tumor and immune cells (CPS),2022-03-16,2022-04-07,2022-04-08,No interpretation given in report,NaN,NaN,NaN,...,Dako PD-L1 IHC 22C3 pharmDx (Keytruda companio...,22C3,NaN,NaN,10-19,2022-04-08,2022-04-06,2.0,0,0


In [173]:
# Among PDL1 tested patients, select highest percent staining for those with repeat testing, merge with nonduplciates, then pivot. 
biomarker_pdl1_status = (
    biomarker_pdl1
    .sort_values(by = ['PatientID', 'bio_status'], ascending = False)
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .pivot(index = 'PatientID', columns = 'BiomarkerName', values = 'bio_status')
    .rename(columns = {'PDL1': 'pdl1_status'})
    .reset_index()
)
biomarker_pdl1_status.columns.name = None

In [174]:
biomarker_pdl1_status.sample(3)

,PatientID,pdl1_status
567,F6E9E9B82AC66,0
985,FB9B732C0D5EF,2
893,FA85D52E6A751,0


In [175]:
row_ID(biomarker_pdl1_status)

(1342, 1342)

In [176]:
biomarker_pdl1_status_wide = (
    pd.concat(
        [biomarker_pdl1_status, 
         pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(biomarker_pdl1_staining['PatientID'])].to_frame(name='PatientID')],
        ignore_index=True,  # This ensures that the index is reset after concatenation
        sort=False
    )
    .fillna(0)
)

In [177]:
row_ID(biomarker_pdl1_status_wide)

(6461, 6461)

In [178]:
biomarker_pdl1_merged = pd.merge(biomarker_pdl1_staining_wide, biomarker_pdl1_status_wide, on = 'PatientID')

In [179]:
biomarker_pdl1_merged.sample(3)

,PatientID,pdl1_staining,pdl1_status
5919,F0313532FDBB2,Missing,0.0
4418,F0470133C131C,Missing,0.0
4814,F291774595DCE,Missing,0.0


In [180]:
# If PDL1 staining is unknown, set to >=1% if ever positive and 0% if ever negative. 
# If PDL1 staining is known, set to >=1% if staining 1-100% and 0% if 0%.
conditions = [
    ((biomarker_pdl1_merged['pdl1_staining'] == 'Missing') & (biomarker_pdl1_merged['pdl1_status'] == 2)) | 
    (biomarker_pdl1_merged['pdl1_staining'] == '>=1%'),
    ((biomarker_pdl1_merged['pdl1_staining'] == 'Missing') & (biomarker_pdl1_merged['pdl1_status'] == 1)) | 
    (biomarker_pdl1_merged['pdl1_staining'] == '0%'), 
    ((biomarker_pdl1_merged['pdl1_staining'] == 'Missing') & (biomarker_pdl1_merged['pdl1_status'] == 0)),
]

choices = ['>=1%', '0%', 'unknown']

biomarker_pdl1_merged.loc[:, 'pdl1_n'] = np.select(conditions, choices, default='unknown').astype(str)

In [181]:
biomarker_pdl1_merged['pdl1_n'].value_counts()

pdl1_n
unknown    5567
>=1%        498
0%          396
Name: count, dtype: int64

In [182]:
biomarker_pdl1_wide = (
    biomarker_pdl1_merged[['PatientID', 'pdl1_n']]
)

In [183]:
row_ID(biomarker_pdl1_wide)

(6461, 6461)

In [184]:
biomarker_wide = pd.merge(biomarker_fgfr_wide, biomarker_pdl1_wide, on = 'PatientID')

In [185]:
row_ID(biomarker_wide)

(6461, 6461)

In [186]:
biomarker_wide.sample(3)

,PatientID,FGFR,pdl1_n
5427,F8018273ACE1E,Missing,unknown
1953,FB3BFAE9E2754,Missing,unknown
1109,FF1FB78C306BC,Positive,0%


In [187]:
%whos DataFrame

Variable                       Type         Data/Info
-----------------------------------------------------
biomarker_fgfr                 DataFrame              PatientID Bioma<...>n[1657 rows x 23 columns]
biomarker_fgfr_wide            DataFrame              PatientID      <...>\n[6461 rows x 2 columns]
biomarker_pdl1                 DataFrame              PatientID Bioma<...>n[2157 rows x 24 columns]
biomarker_pdl1_merged          DataFrame              PatientID pdl1_<...>\n[6461 rows x 4 columns]
biomarker_pdl1_staining        DataFrame              PatientID pdl1_<...>\n[1342 rows x 2 columns]
biomarker_pdl1_staining_wide   DataFrame              PatientID pdl1_<...>\n[6461 rows x 2 columns]
biomarker_pdl1_status          DataFrame              PatientID  pdl1<...>\n[1342 rows x 2 columns]
biomarker_pdl1_status_wide     DataFrame              PatientID  pdl1<...>\n[6461 rows x 2 columns]
biomarker_pdl1_wide            DataFrame              PatientID   pdl<...>\n[6461 rows x 2 c

In [188]:
# Keep biomarker_wide, demographics, enhanced_adv
del biomarker_fgfr
del biomarker_fgfr_wide
del biomarker_pdl1
del biomarker_pdl1_merged
del biomarker_pdl1_staining
del biomarker_pdl1_staining_wide
del biomarker_pdl1_status
del biomarker_pdl1_status_wide
del biomarker_pdl1_wide 
del biomarker_win
del biomarkers

In [189]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
biomarker_wide   DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cohort           DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics     DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
enhanced_adv     DataFrame              PatientID diagn<...>n[6461 rows x 16 columns]


5. Clean ECOG dataset

In [190]:
ecog = pd.read_csv('../data/ECOG.csv')

In [191]:
ecog = ecog[ecog['PatientID'].isin(cohort_IDs)]

In [192]:
row_ID(ecog)

(118838, 5453)

The ECOG data, like the biomarker data, is in long format with multiple entries per patient and needs to be converted to wide, with one ECOG score per patient. The index data of interest here is the start_date of treatment. ECOG score closest to the index date (and within -90 to + 30 days of treatment start) will be used. In the case of two ECOG scores on the same day or equidistant but on opposite sides of the index date, the higher ECOG score (worse performance) will be selected.

Note as before, BaselineECOG will not be used because there can be multiple entries.

In [193]:
ecog = pd.merge(ecog, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [194]:
ecog.sample(3)

,PatientID,PracticeID,EcogDate,EcogValue,StartDate
103271,FDEF719C4230A,S291CF564BD92,2015-03-12,1,2014-05-08
21606,F3DC7612A126B,S507DCE5DD8C6,2019-11-21,2,2018-09-12
102858,F3B9C324B1C0C,S291CF564BD92,2018-01-22,2,2017-11-20


In [195]:
ecog['EcogDate'] = pd.to_datetime(ecog['EcogDate'], format="%Y-%m-%d")
ecog['StartDate'] = pd.to_datetime(ecog['StartDate'], format="%Y-%m-%d")

In [196]:
ecog.loc[:, 'ecog_date_diff'] = (ecog['EcogDate'] - ecog['StartDate']).dt.days

In [197]:
row_ID(ecog)

(118838, 5453)

In [198]:
ecog_win = ecog[(ecog['ecog_date_diff'] >= -90) & (ecog['ecog_date_diff'] <= 30)]

In [199]:
row_ID(ecog_win)

(20722, 4842)

In [200]:
ecog_win['ecog_date_diff'].min()

np.int64(-90)

In [201]:
#convert ecog_date_diff to absolute value to be able to identify value closest to start date
ecog_win.loc[:, 'ecog_date_diff'] = ecog_win['ecog_date_diff'].abs()

In [202]:
ecog_win['ecog_date_diff'].min()

np.int64(0)

In [203]:
# Sort values with ECOG nearest to time of diagnosis as top row (and largest ECOG if multiple ECOGs that day) then select top row.   ECOG date nearest to day of diagnosis as top row and largest ES
ecog_diagnosis_wide = (
    ecog_win
    .sort_values(by = ['PatientID', 'ecog_date_diff', 'EcogValue'], ascending = [True, True, False])
    .drop_duplicates(subset = ['PatientID'], keep = 'first' )
    .filter(items = ['PatientID', 'EcogValue'])
    .rename(columns = {'EcogValue': 'ecog_diagnosis'})
)

In [204]:
row_ID(ecog_diagnosis_wide)

(4842, 4842)

In [205]:
# Append missing IDs. 
ecog_diagnosis_wide = (
    pd.concat(
        [ecog_diagnosis_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(ecog_diagnosis_wide['PatientID'])].to_frame(name='PatientID')],
        sort = False)
    .fillna('unknown')
)

In [206]:
row_ID(ecog_diagnosis_wide)

(6461, 6461)

In [207]:
ecog_diagnosis_wide.value_counts('ecog_diagnosis')

ecog_diagnosis
1.0        2185
unknown    1619
0.0        1615
2.0         805
3.0         227
4.0          10
Name: count, dtype: int64

In [208]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cohort                DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics          DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
ecog                  DataFrame                PatientID    <...>[118838 rows x 6 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6461 rows x 2 columns]
ecog_win              DataFrame                PatientID    <...>n[20722 rows x 6 columns]
enhanced_adv          DataFrame              PatientID diagn<...>n[6461 rows x 16 columns]


In [209]:
del ecog
del ecog_win

6. Clean Vitals dataset

In [210]:
vitals = pd.read_csv('../data/Vitals.csv')

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/446095882.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  vitals = pd.read_csv('../data/Vitals.csv')


In [211]:
row_ID(vitals)

(3604484, 13109)

In [212]:
vitals = vitals[vitals['PatientID'].isin(cohort_IDs)]

In [213]:
row_ID(vitals)

(2038026, 6461)

In [214]:
vitals.value_counts('Test')

Test
heart rate                                                       371278
body temperature                                                 338654
systolic blood pressure                                          260710
diastolic blood pressure                                         254910
body weight                                                      240184
oxygen saturation in arterial blood by pulse oximetry            206556
body height                                                      204827
respiratory rate                                                 102846
pain severity - 0-10 verbal numeric rating [score] - reported     31766
body surface area                                                 14625
body mass index (bmi) [ratio]                                     11668
head occipital-frontal circumference by tape measure                  2
Name: count, dtype: int64

The Vitals dataset is also in long format, with multiple entries per patient. We will obtain weights, percent change in weight, weight slope, BMI, BP and HR for patients. Index date will be the start date, and the eligibility window will be -90 days to +30 days from the start date. In the case of two weights on the same day or equidistant but on opposite sides of the index date, the lowest weight will be selected. Percent change in weight and weight slope 3 months within metastatic diagnosis will be calculated. Patients must have at least two weight recordings to calculate percent change in weight or weight slope.

Weight and BMI

In [215]:
#Create weight dataframe; remove weight values that are empty or equal to zero
weight = (
    vitals
    .query('Test == "body weight"')
    .filter(items = ['PatientID', 'TestDate', 'TestResultCleaned'])
    .rename(columns = {'TestResultCleaned': 'weight'})
    .dropna(subset = ['weight'])
    .query('weight != 0')
)

In [216]:
weight['TestDate'] = pd.to_datetime(weight['TestDate'], format="%Y-%m-%d")

In [217]:
weight = pd.merge(weight, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [218]:
weight['StartDate'] = pd.to_datetime(weight['StartDate'], format="%Y-%m-%d")

In [219]:
# Weight elgibliity window is -90 and +30 from treatment Start Date. 
weight_win_bmi = (
    weight
    .assign(weight_date_diff = (weight['TestDate'] - weight['StartDate']).dt.days)
    .query('weight_date_diff >= -90 and weight_date_diff <= 30')
)

In [220]:
weight_win_bmi['weight_date_diff'].min()

np.int64(-90)

In [221]:
weight_win_bmi.loc[:, 'weight_date_diff'] = weight_win_bmi['weight_date_diff'].abs()
weight_win_bmi['weight_date_diff'].min()

np.int64(0)

In [222]:
# Select weight closest to Start Date; lowest weight selected in the event of two weights on same day or equidistant. 
weight_bmi_wide = (
    weight_win_bmi
    .sort_values(by = ['PatientID', 'weight_date_diff', 'weight'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'weight'])
    .rename(columns = {'weight': 'weight_diag'})
)

In [223]:
row_ID(weight_bmi_wide)

(6332, 6332)

In [224]:
# Dataframe of average height for each patient. 
height_avg = (
    vitals
    .query('Test == "body height"')
    .filter(items = ['PatientID', 'TestResultCleaned'])
    .query('TestResultCleaned > 0')
    .groupby('PatientID')['TestResultCleaned'].mean()
    .to_frame()
    .reset_index()
    .rename(columns = {'TestResultCleaned': 'height_avg'})
)

In [225]:
weight_bmi_wide = pd.merge(weight_bmi_wide, height_avg, on = 'PatientID', how = 'left')

In [226]:
weight_bmi_wide.sample(3)

,PatientID,weight_diag,height_avg
1552,F3E8F6B597FAA,57.243310,158.669818
2671,F6D83677AD665,59.499364,174.509190
35,F0149A4DE845F,94.800728,175.260000


In [227]:
# Create BMI column. 
weight_bmi_wide = (
    weight_bmi_wide
    .assign(bmi_diag = lambda x: (x['weight_diag']/(x['height_avg']*x['height_avg']))*10000)
    .drop(columns = ['height_avg'])
)

In [228]:
weight_bmi_wide.sample(3)
row_ID(weight_bmi_wide)

(6332, 6332)

In [229]:
# Append excluded IDs from cohort and create a missing variable for those without BMI at diagnosis. 
weight_bmi_wide = (
    pd.concat(
        [weight_bmi_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(weight_bmi_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
    .fillna('unknown')
)

Percent Change in weight

In [230]:
# Select elgbility window of -90 to +90 days from advanced diagnosis.
weight_win_summary = (
    weight
    .assign(weight_date_diff = (weight['TestDate'] - weight['StartDate']).dt.days)
    .query('weight_date_diff >= -90 and weight_date_diff <= 90')
)

In [231]:
row_ID(weight_win_summary)

(70735, 6339)

In [232]:
# Select patients with more than 1 weight recording within elgibility window.
weight_win_summary = weight_win_summary[weight_win_summary.duplicated(subset = ['PatientID'], keep = False)]

In [233]:
row_ID(weight_win_summary)

(70651, 6255)

In [234]:
# Select weight from the earliest time within elgibility window. 
weight_tmin = weight_win_summary.loc[weight_win_summary.groupby('PatientID')['weight_date_diff'].idxmin()]

In [235]:
# Select weight from the latest time within elgibility window. 
weight_tmax = weight_win_summary.loc[weight_win_summary.groupby('PatientID')['weight_date_diff'].idxmax()]

In [236]:
# Combine above two dataframes and sort from earliest recorded weight to latest recorded weight for each patient. 
weight_tcomb = (
    pd.concat([weight_tmin, weight_tmax])
    .sort_values(by = ['PatientID', 'weight_date_diff'], ascending = True)
)

In [237]:
row_ID(weight_tcomb)

(12510, 6255)

In [238]:
weight_tcomb.loc[:, 'weight_pct_change'] = weight_tcomb.groupby('PatientID')['weight'].pct_change()

In [239]:
weight_tcomb.loc[:, 'diff_date_diff'] = weight_tcomb['weight_date_diff'].diff()

In [240]:
# Drop empty rows for weight_pct_change.
weight_pct_wide = (
    weight_tcomb
    .dropna(subset = ['weight_pct_change'])
    .filter(items = ['PatientID', 'weight_pct_change', 'diff_date_diff'])
)

In [241]:
weight_pct_wide.sample(3)

,PatientID,weight_pct_change,diff_date_diff
81652,F5914B5D88997,-0.078431,85.0
20008,F16940B52BC98,0.030457,162.0
26885,F1E7346FB0CB0,-0.041017,134.0


In [242]:
row_ID(weight_pct_wide)

(6255, 6255)

In [243]:
weight_pct_wide = (
    pd.concat(
        [weight_pct_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(weight_pct_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
    .fillna('unknown')
)

In [244]:
row_ID(weight_pct_wide)

(6461, 6461)

In [245]:
weight_pct_wide.sample(3)

,PatientID,weight_pct_change,diff_date_diff
169053,FB9BDD7DD061F,-0.094315,35.0
141146,F9CFAFC5C0E7D,-0.065606,174.0
91495,F6600A3ECE370,0.001634,150.0


In [246]:
weight_pct_wide.loc[:, 'weight_pct_na'] = np.where(weight_pct_wide['weight_pct_change'].isna(), 1, 0)

Weight Slope

In [247]:
from scipy.stats import linregress 

In [248]:
weight_win_summary.sample(3)

,PatientID,TestDate,weight,StartDate,weight_date_diff
173420,FBFE3A4B75F4E,2017-06-21,62.686414,2017-05-18,34
96675,F6C087824D9B4,2016-02-10,96.252222,2016-04-25,-75
168352,FB9544172B4FC,2013-02-18,42.864444,2013-02-04,14


In [249]:
import datetime as dt
weight_win_summary.loc[:, 'date_ordinal'] = weight_win_summary['TestDate'].map(dt.datetime.toordinal)

In [250]:
#To calculate slope, need to isolate cases where there is more than one date_ordinal entry
filtered_weight_win_summary = (
    weight_win_summary
    .groupby('PatientID')
    .filter(lambda x: x['date_ordinal'].nunique() > 1)
)
row_ID(filtered_weight_win_summary)

(70642, 6251)

In [251]:
# Dataframe of slope for weight recordings within window period (kg/day).
weight_slope_wide = (
    filtered_weight_win_summary
    .groupby('PatientID')
    .apply(lambda x: pd.Series(linregress(x['date_ordinal'], x['weight'])))
    .rename(columns = {0: 'weight_slope'})
    .reset_index()
    .filter(items = ['PatientID', 'weight_slope']))

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/2912839740.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series(linregress(x['date_ordinal'], x['weight'])))


In [252]:
weight_slope_wide.sample(3)

,PatientID,weight_slope
3388,F8CFC17530A36,-0.054125
3737,F9A292CB00663,-0.002801
28,F011D2DC992BD,0.012730


In [253]:
row_ID(weight_slope_wide)

(6251, 6251)

In [254]:
#Append missing cohort IDs
weight_slope_wide = (
    pd.concat(
        [weight_slope_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(weight_slope_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
    .fillna('unknown')
)

In [255]:
row_ID(weight_slope_wide)

(6461, 6461)

Weight merge

In [256]:
weight_wide = pd.merge(weight_bmi_wide, weight_pct_wide, on = 'PatientID')

In [257]:
weight_wide = pd.merge(weight_wide, weight_slope_wide, on = 'PatientID')

In [258]:
row_ID(weight_wide)

(6461, 6461)

In [259]:
weight_wide.sample(3)

,PatientID,weight_diag,bmi_diag,weight_pct_change,diff_date_diff,weight_pct_na,weight_slope
5043,FCC3CCED5DF54,83.460928,34.76611,0.00882,110.0,0,0.009691
2375,F60932F05D3BE,56.517563,20.110743,0.014815,118.0,0,0.018422
2223,F5A34F55D2044,46.447821,18.728997,0.051887,173.0,0,0.016926


In [260]:
weight_wide = weight_wide.drop(columns = ['diff_date_diff'])

In [261]:
weight_wide.sample(3)

,PatientID,weight_diag,bmi_diag,weight_pct_change,weight_pct_na,weight_slope
5222,FD32E3DA1D101,72.57472,31.247511,0.00625,0,0.016517
2441,F6356C3513BE4,88.541158,29.67973,-0.095046,0,-0.082773
4064,FA4A8531B8AA5,78.471416,24.82264,-0.104046,0,-0.130986


Blood pressure

In [262]:
sbp = (
    vitals
    .query('Test == "systolic blood pressure"')
    .filter(items = ['PatientID', 'TestDate', 'TestResult'])
    .rename(columns = {'TestResult': 'sbp'})
    .dropna(subset = ['sbp'])
    .query('sbp != 0')
)

In [263]:
sbp.sample(3)

,PatientID,TestDate,sbp
2869704,FCC5CFF5D0011,2015-01-28,150
875337,F3E5D35BC3607,2022-09-06,117
46484,F035178A7B120,2021-01-27,148


In [264]:
sbp = pd.merge(sbp, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [265]:
sbp['TestDate'] = pd.to_datetime(sbp['TestDate'], format="%Y-%m-%d")
sbp['StartDate'] = pd.to_datetime(sbp['StartDate'], format="%Y-%m-%d")

In [266]:
sbp.sample(3)

,PatientID,TestDate,sbp,StartDate
69226,F44095CD7493F,2017-04-03,137,2019-04-23
247185,FF44D92E2B3BC,2018-12-03,102,2018-02-23
143317,F8FC9CC70886C,2015-07-31,135,2016-09-02


In [267]:
sbp_win = (
    sbp
    .assign(sbp_date_diff = (sbp['TestDate'] - sbp['StartDate']).dt.days)
    .query('sbp_date_diff >= -90 and sbp_date_diff <= 30')
)

In [268]:
sbp_win.loc[:, 'sbp_date_diff'] = sbp_win['sbp_date_diff'].abs()

In [269]:
sbp_wide = (
    sbp_win
    .sort_values(by = ['PatientID', 'sbp_date_diff', 'sbp'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'sbp'])
    .rename(columns = {'sbp': 'sbp_diag'})
)

In [270]:
sbp_wide.sample(3)

,PatientID,sbp_diag
27464,F1C365C98E575,109
136284,F89392E350B58,158
877,F010518E6483E,140


In [271]:
row_ID(sbp_wide)

(5824, 5824)

In [272]:
row_ID(sbp)

(259897, 5861)

In [273]:
#Append missing cohort IDs
sbp_wide = (
    pd.concat(
        [sbp_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(sbp_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [274]:
row_ID(sbp_wide)

(6461, 6461)

Diastolic Blood Pressure

In [275]:
dbp = (
    vitals
    .query('Test == "diastolic blood pressure"')
    .filter(items = ['PatientID', 'TestDate', 'TestResult'])
    .rename(columns = {'TestResult': 'dbp'})
    .dropna(subset = ['dbp'])
    .query('dbp != 0')
)

row_ID(dbp)

(254176, 5862)

In [276]:
dbp = pd.merge(dbp, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [277]:
dbp['TestDate'] = pd.to_datetime(dbp['TestDate'], format="%Y-%m-%d")
dbp['StartDate'] = pd.to_datetime(dbp['StartDate'], format="%Y-%m-%d")

In [278]:
dbp.sample(3)

,PatientID,TestDate,dbp,StartDate
249286,FFB01CB4D1E8E,2008-09-16,68,2012-04-25
137850,F8DAAC355564D,2015-07-24,95,2022-03-31
117859,F7878116D9FA8,2016-12-15,76,2015-02-17


In [279]:
dbp_win = (
    dbp
    .assign(dbp_date_diff = (dbp['TestDate'] - dbp['StartDate']).dt.days)
    .query('dbp_date_diff >= -90 and dbp_date_diff <= 30')
)

In [280]:
dbp_win.loc[:, 'dbp_date_diff'] = dbp_win['dbp_date_diff'].abs()

In [281]:
dbp_wide = (
    dbp_win
    .sort_values(by = ['PatientID', 'dbp_date_diff', 'dbp'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'dbp'])
    .rename(columns = {'dbp': 'dbp_diag'})
)

In [282]:
dbp_wide.sample(3)

,PatientID,dbp_diag
11357,F0B766374377B,83
7691,F07411C7D98DC,68
62608,F3E1CC3AE023C,78


In [283]:
#Append missing cohort IDs
dbp_wide = (
    pd.concat(
        [dbp_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(dbp_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [284]:
row_ID(dbp_wide)

(6461, 6461)

Heart Rate

In [285]:
hr = (
    vitals
    .query('Test == "heart rate"')
    .filter(items = ['PatientID', 'TestDate', 'TestResult'])
    .rename(columns = {'TestResult': 'hr'})
    .dropna(subset = ['hr'])
    .query('hr != 0')
)

In [286]:
row_ID(hr)

(370615, 6436)

In [287]:
hr = pd.merge(hr, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [288]:
hr['TestDate'] = pd.to_datetime(hr['TestDate'], format="%Y-%m-%d")
hr['StartDate'] = pd.to_datetime(hr['StartDate'], format="%Y-%m-%d")

In [289]:
hr.sample(3)

,PatientID,TestDate,hr,StartDate
200531,F8BB8ED38D7C4,2022-08-27,87,2023-06-19
369621,FFF4FD17F5326,2022-04-08,71,2021-07-20
219145,F97FD429ADCAC,2018-04-17,83,2019-04-08


In [290]:
hr_win = (
    hr
    .assign(hr_date_diff = (hr['TestDate'] - hr['StartDate']).dt.days)
    .query('hr_date_diff >= -90 and hr_date_diff <= 30')
)

In [291]:
hr_win.loc[:, 'hr_date_diff'] = hr_win['hr_date_diff'].abs()

In [292]:
hr_wide = (
    hr_win
    .sort_values(by = ['PatientID', 'hr_date_diff', 'hr'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'hr'])
    .rename(columns = {'hr': 'hr_diag'})
)

In [293]:
hr_wide.sample(3)

,PatientID,hr_diag
165626,F74C635137858,95
142001,F63F7900A89C6,75
14739,F0A251021E3D5,99


In [294]:
#Append missing cohort IDs
hr_wide = (
    pd.concat(
        [hr_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(hr_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [295]:
row_ID(hr_wide)

(6461, 6461)

Pulse Oximetry

In [296]:
pulse_ox = (
    vitals
    .query('Test == "oxygen saturation in arterial blood by pulse oximetry"')
    .filter(items = ['PatientID', 'TestDate', 'TestResult'])
    .rename(columns = {'TestResult': 'pulse_ox'})
    .dropna(subset = ['pulse_ox'])
    .query('pulse_ox != 0')
)

In [297]:
row_ID(pulse_ox)

(203416, 4571)

In [298]:
pulse_ox = pd.merge(pulse_ox, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [299]:
pulse_ox['TestDate'] = pd.to_datetime(pulse_ox['TestDate'], format="%Y-%m-%d")
pulse_ox['StartDate'] = pd.to_datetime(pulse_ox['StartDate'], format="%Y-%m-%d")

In [300]:
pulse_ox.sample(3)

,PatientID,TestDate,pulse_ox,StartDate
42445,F351225E424B8,2021-09-08,99,2021-09-08
47967,F3B9AB8134289,2017-12-05,96,2017-12-06
76904,F5EC6624D300A,2022-09-14,95,2022-04-11


In [301]:
pulse_ox_win = (
    pulse_ox
    .assign(pulse_ox_date_diff = (pulse_ox['TestDate'] - pulse_ox['StartDate']).dt.days)
    .query('pulse_ox_date_diff >= -90 and pulse_ox_date_diff <= 30')
)

In [302]:
pulse_ox_win.loc[:, 'pulse_ox_date_diff'] = pulse_ox_win['pulse_ox_date_diff'].abs()

In [303]:
pulse_ox_wide = (
    pulse_ox_win
    .sort_values(by = ['PatientID', 'pulse_ox_date_diff', 'pulse_ox'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'pulse_ox'])
    .rename(columns = {'pulse_ox': 'pulse_ox_diag'})
)

In [304]:
pulse_ox_wide.sample(3)

,PatientID,pulse_ox_diag
173912,FDA360FE40724,98
31865,F28FA83B37CF7,98
122915,F98F7FFE152E7,94


In [305]:
#Append missing cohort IDs
pulse_ox_wide = (
    pd.concat(
        [pulse_ox_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(pulse_ox_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [306]:
row_ID(pulse_ox_wide)

(6461, 6461)

Body temperature

In [307]:
vitals['TestUnits'].value_counts()

TestUnits
degree Fahrenheit    218219
in                   193960
lb                   185107
%                    121343
mm Hg                 64102
bpm                   41122
oz                    35325
degree Celsius        29619
kg                    15089
cm                    10784
m2                     1926
min                     317
breaths/min              92
ft                       28
Name: count, dtype: int64

In [308]:
temp = (
    vitals
    .query('Test == "body temperature"')
    .filter(items = ['PatientID', 'TestDate', 'TestResult', 'TestUnits'])
    .rename(columns = {'TestResult': 'temp_raw'})
    .dropna(subset = ['temp_raw'])
    .query('temp_raw != 0')
)

In [309]:
temp['TestUnits'].value_counts()

TestUnits
degree Fahrenheit    217672
degree Celsius        28808
Name: count, dtype: int64

In [310]:
temp.sample(3)

,PatientID,TestDate,temp_raw,TestUnits
2338418,FA5F67F734E8D,2022-05-19,98.6,degree Fahrenheit
2626813,FBA609E20A069,2012-05-10,99.5,degree Fahrenheit
2755054,FC3EBEA41CCC0,2022-03-18,97.4,degree Fahrenheit


In [311]:
temp['temp_raw'] = pd.to_numeric(temp['temp_raw'], errors='coerce')

temp['temp'] = (
    np.where(temp['TestUnits'] == 'degree Celsius', (temp['temp_raw']*9/5) + 32, temp['temp_raw'])
)

In [312]:
row_ID(temp)

(336908, 6276)

In [313]:
temp = temp[temp['TestUnits'].isin(['degree Celsius', 'degree Fahrenheit'])]

In [314]:
row_ID(temp)

(246480, 5743)

In [315]:
temp = pd.merge(temp, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [316]:
temp['TestDate'] = pd.to_datetime(temp['TestDate'], format="%Y-%m-%d")
temp['StartDate'] = pd.to_datetime(temp['StartDate'], format="%Y-%m-%d")

In [317]:
temp.sample(3)

,PatientID,TestDate,temp_raw,TestUnits,temp,StartDate
42422,F2BF270C5885D,2015-06-15,36.9,degree Celsius,98.42,2013-12-03
127954,F8841D535D792,2017-09-27,36.9,degree Celsius,98.42,2017-10-16
44495,F2DDF26FC5E37,2020-06-17,97.5,degree Fahrenheit,97.50,2020-06-17


In [318]:
temp_win = (
    temp
    .assign(temp_date_diff = (temp['TestDate'] - temp['StartDate']).dt.days)
    .query('temp_date_diff >= -90 and temp_date_diff <= 30')
)

In [319]:
temp_win.loc[:, 'temp_date_diff'] = temp_win['temp_date_diff'].abs()

In [320]:
temp_wide = (
    temp_win
    .sort_values(by = ['PatientID', 'temp_date_diff', 'temp'], ascending = [True, True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'temp'])
    .rename(columns = {'temp': 'temp_diag'})
)

In [321]:
temp_wide.sample(3)

,PatientID,temp_diag
240206,FF97758679E0F,97.5
86643,F590DFAF69FCF,97.8
58574,F3C7ABBB3A3DF,98.0


In [322]:
row_ID(temp_wide)

(5625, 5625)

In [323]:
#Append missing cohort IDs
temp_wide = (
    pd.concat(
        [temp_wide,
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(temp_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [324]:
row_ID(temp_wide)

(6461, 6461)

In [325]:
vitals_wide = pd.merge(sbp_wide, dbp_wide, on = 'PatientID', how = 'left')

In [326]:
vitals_wide = pd.merge(vitals_wide, hr_wide, on = 'PatientID', how = 'left')

In [327]:
vitals_wide = pd.merge(vitals_wide, pulse_ox_wide, on = 'PatientID', how = 'left')

In [328]:
vitals_wide = pd.merge(vitals_wide, temp_wide, on = 'PatientID', how = 'left')

In [329]:
vitals_wide.sample(3)

,PatientID,sbp_diag,dbp_diag,hr_diag,pulse_ox_diag,temp_diag
1200,F346718C58D97,120,74,76,98,97.9
6213,F4B0C44A47EFB,NaN,NaN,99,NaN,NaN
723,F1F2CFB204131,106,62,61,98,97.2


In [330]:
row_ID(vitals_wide)

(6461, 6461)

In [331]:
%whos DataFrame

Variable                      Type         Data/Info
----------------------------------------------------
biomarker_wide                DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cohort                        DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
dbp                           DataFrame                PatientID   T<...>[254176 rows x 4 columns]
dbp_wide                      DataFrame              PatientID dbp_d<...>\n[6461 rows x 2 columns]
dbp_win                       DataFrame                PatientID   T<...>n[46322 rows x 5 columns]
demographics                  DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
ecog_diagnosis_wide           DataFrame                PatientID eco<...>\n[6461 rows x 2 columns]
enhanced_adv                  DataFrame              PatientID diagn<...>n[6461 rows x 16 columns]
filtered_weight_win_summary   DataFrame                PatientID   T<...>n[70642 rows x 6 columns]
hei

In [332]:
del dbp
del dbp_wide
del dbp_win
del filtered_weight_win_summary
del height_avg
del hr
del hr_wide
del hr_win
del pulse_ox
del pulse_ox_wide
del pulse_ox_win
del sbp
del sbp_wide
del sbp_win
del temp
del temp_wide
del temp_win
del vitals
del weight
del weight_bmi_wide
del weight_pct_wide
del weight_slope_wide
del weight_tcomb
del weight_tmax
del weight_tmin
del weight_win_bmi
del weight_win_summary

In [333]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cohort                DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics          DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6461 rows x 2 columns]
enhanced_adv          DataFrame              PatientID diagn<...>n[6461 rows x 16 columns]
vitals_wide           DataFrame              PatientID sbp_d<...>\n[6461 rows x 6 columns]
weight_wide           DataFrame              PatientID weigh<...>\n[6461 rows x 6 columns]


7. Clean labs dataset

In [334]:
lab = pd.read_csv('../data/Lab.csv')

In [335]:
row_ID(lab)

(9373598, 12700)

In [336]:
lab = lab[lab['PatientID'].isin(cohort_IDs)]

In [337]:
row_ID(lab)

(5615579, 6408)

In [338]:
lab.sample(3)

,PatientID,PracticeID,TestDate,LOINC,Test,LabComponent,TestBaseName,LabSource,TestUnits,TestUnitsCleaned,ResultDate,TestResult,TestResultCleaned,MinNorm,MinNormCleaned,MaxNorm,MaxNormCleaned
4924908,F8941B4EC5940,SC3260C6286CA,2016-01-06,26449-9,eosinophils [#/volume] in blood,Eosinophils [#/?volume] in Blood,eosinophils,source,10*3/uL,10*9/L,2016-01-06,0.0,0.0,0.0,0.0,0.6,0.6
4872534,F880028548386,S98D0ED5E3A94,2019-12-18,26450-7,eosinophils/100 leukocytes in blood,Eosinophils/?100 leukocytes in Blood,eosinophils/100 leukocytes,source,%,%,2019-12-18,0.8,0.8,0.7,0.7,6.0,6.0
7805031,FD64C4908D72A,SC3260C6286CA,2023-10-24,1742-6,alanine aminotransferase [enzymatic activity/v...,Alanine aminotransferase (ALT or SGPT),alanine aminotransferase,source,U/L,U/L,2023-10-25,84,84.0,0.0,0.0,31.0,31.0


In [339]:
lab['TestBaseName'].value_counts()

TestBaseName
hematocrit                              220989
platelets                               219831
hemoglobin                              214562
leukocytes                              213181
erythrocytes                            210675
                                         ...  
human papilloma virus 16 ag                  1
acarboxyprothrombin                          1
cortisol^1h post dose corticotropin          1
cortisol^30m post dose corticotropin         1
cortisol.free/cortisol.total                 1
Name: count, Length: 155, dtype: int64

The lab table is in long format, and we will have to convert to wide with the table including the following NCCN recommended labs:

Creatinine -- (LOINC: 2160-0 and 38483-4)
Hemoglobin -- (LOINC: 718-7 and 20509-6)
White blood cell count -- (LOINC: 26464-8 and 6690-2)
Neutrophil count -- (LOINC: 26499-4, 751-8, 30451-9, and 753-4)
Albumin, serum -- (LOINC: 1751-7)
Total bilirubin -- (LOINC: 42719-5 and 1975-2)
Sodium — (LOINC: 2947-0 and 2951-2)
Bicarb — (LOINC: 1963-8, 1959-6, 14627-4, 1960-4, and 2028-9)
Calcium — (LOINC: 17861-6 and 49765-1)
AST — (LOINC: 1920-8)
ALT — (LOINC: 1742-6, 1743-4, and 1744-2)
Platelet -- (LOINC: 26515-7, 777-3, 778-1, and 49497-1)
Potassium -- (LOINC: 6298-4 and 2823-3)
Chloride -- (LOINC: 2075-0)
BUN -- (LOINC: 3094-0)
ALP -- (LOINC: 6768-6)

In addition I will also capture the lymphocyte count as the neutrophil/lymphocyte ratio has been shown to be associated with response to checkpoint inhibitor for some malignancies.

Lymphocyte count -- (LOINC: 26474-7, 30364-4, 731-0, 732-8)

The index date will be time of therapy start date with an elgibility window of -90 days to +30 days. The lab value closest to the index date will be selected for each patient. The following summary statistics, using an elgibility window of negative infinity to +30 days from advanced diagnosis, will also be created for the above variables:

Max
Min
Mean
Standard deviation
Slope

Baseline Lab Values

In [340]:
lab = pd.merge(lab, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [341]:
lab.sample(3)

,PatientID,PracticeID,TestDate,LOINC,Test,LabComponent,TestBaseName,LabSource,TestUnits,TestUnitsCleaned,ResultDate,TestResult,TestResultCleaned,MinNorm,MinNormCleaned,MaxNorm,MaxNormCleaned,StartDate
771268,F244380231510,SC3260C6286CA,2012-05-03,19123-9,magnesium [mass/volume] in serum or plasma,Magnesium,magnesium,source,mg/dL,mg/dL,2012-05-04,1.6,1.6,1.6,1.6,2.6,2.6,2012-01-05
753185,F239AA6A7F5FC,SC3260C6286CA,2012-04-23,2028-9,"carbon dioxide, total [moles/volume] in serum ...",Carbon dioxide,carbon dioxide,source,meq/L,NaN,2012-04-24,23,NaN,22.0,NaN,33.0,NaN,2012-01-06
2605223,F78776A0335A7,S98D0ED5E3A94,2014-07-30,2075-0,chloride [moles/volume] in serum or plasma,Chloride,chloride,source,mmol/L,mmol/L,2014-07-30,95,95.0,94.0,94.0,112.0,112.0,2014-06-30


In [342]:
lab['ResultDate'] = pd.to_datetime(lab['ResultDate'], format="%Y-%m-%d")

In [343]:
# Select rows with clinically relevant labs
lab_core = (
    lab[
    (lab['LOINC'] == "2160-0") |
    (lab['LOINC'] == "38483-4") | 
    (lab['LOINC'] == "718-7") |
    (lab['LOINC'] == "20509-6") |
    (lab['LOINC'] == "26464-8") |
    (lab['LOINC'] == "6690-2") |
    (lab['LOINC'] == "26499-4") |
    (lab['LOINC'] == "751-8") |
    (lab['LOINC'] == "30451-9") |
    (lab['LOINC'] == "753-4") |
    (lab['LOINC'] == "1751-7") |
    (lab['LOINC'] == "42719-5") |
    (lab['LOINC'] == "1975-2") |
    (lab['LOINC'] == "2947-0") |
    (lab['LOINC'] == "2951-2") |
    (lab['LOINC'] == "1963-8") |
    (lab['LOINC'] == "1959-6") |
    (lab['LOINC'] == "14627-4") |
    (lab['LOINC'] == "1960-4") |
    (lab['LOINC'] == "2028-9") |
    (lab['LOINC'] == "17861-6") |
    (lab['LOINC'] == "49765-1") |
    (lab['LOINC'] == "1920-8") |
    (lab['LOINC'] == "1742-6") | 
    (lab['LOINC'] == "1743-4") |
    (lab['LOINC'] == "1744-2") |
    (lab['LOINC'] == "26515-7") | 
    (lab['LOINC'] == "777-3") |
    (lab['LOINC'] == "778-1") |
    (lab['LOINC'] == "49497-1") | 
    (lab['LOINC'] == "6298-4") |
    (lab['LOINC'] == "2823-3") |
    (lab['LOINC'] == "2075-0") | 
    (lab['LOINC'] == "3094-0") | 
    (lab['LOINC'] == "6768-6") | 
    (lab['LOINC'] == "26474-7")| 
    (lab['LOINC'] == "30364-4")| 
    (lab['LOINC'] == "731-0")  | 
    (lab['LOINC'] == "732-8")]
    .filter(items = ['PatientID', 
                     'ResultDate', 
                     'LOINC', 
                     'LabComponent', 
                     'TestUnits', 
                     'TestUnitsCleaned', 
                     'TestResult', 
                     'TestResultCleaned', 
                     'StartDate'])
)

In [344]:
lab_core.sample(3)

,PatientID,ResultDate,LOINC,LabComponent,TestUnits,TestUnitsCleaned,TestResult,TestResultCleaned,StartDate
1111881,F329C2093A46B,1999-08-09,2075-0,Chloride,mmol/L,mmol/L,103,103.0,2021-04-06
3470004,F9FB0A871A856,2020-01-10,1975-2,"Bilirubin (Total), serum",mg/dL,mg/dL,0.4,0.4,2020-12-22
1963977,F5902C40DC43B,2015-08-13,2951-2,Sodium [Moles/?volume] in Serum or Plasma,mmol/L,mmol/L,138,138.0,2015-08-25


In [345]:
lab_core['LOINC'] = lab_core['LOINC'].astype(str)
conditions = [
    ((lab_core['LOINC'] == '2160-0') | (lab_core['LOINC'] == '38483-4')),
    ((lab_core['LOINC'] == '718-7') | (lab_core['LOINC'] == '20509-6')),
    ((lab_core['LOINC'] == '26464-8') | (lab_core['LOINC'] == '6690-2')), 
    ((lab_core['LOINC'] == '26499-4') | (lab_core['LOINC'] == '751-8') | (lab_core['LOINC'] == '30451-9') | (lab_core['LOINC'] == '753-4')),
    (lab_core['LOINC'] == '1751-7'),
    ((lab_core['LOINC'] == '42719-5') | (lab_core['LOINC'] == '1975-2')),
    ((lab_core['LOINC'] == '2947-0') | (lab_core['LOINC'] == '2951-2')),
    ((lab_core['LOINC'] == '1963-8') | (lab_core['LOINC'] == '1959-6') | (lab_core['LOINC'] == '14627-4') | (lab_core['LOINC'] == '1960-4') | (lab_core['LOINC'] == '2028-9')),
    ((lab_core['LOINC'] == '17861-6') | (lab_core['LOINC'] == '49765-1')),
    (lab_core['LOINC'] == '1920-8'),
    ((lab_core['LOINC'] == '1742-6') | (lab_core['LOINC'] == '1743-4') | (lab_core['LOINC'] == '1744-2')),
    ((lab_core['LOINC'] == '26515-7') | (lab_core['LOINC'] == '777-3') | (lab_core['LOINC'] == '778-1') | (lab_core['LOINC'] == '49497-1')),
    ((lab_core['LOINC'] == '6298-4') | (lab_core['LOINC'] == '2823-3')),
    (lab_core['LOINC'] == '2075-0'), 
    (lab_core['LOINC'] == '3094-0'),
    (lab_core['LOINC'] == '6768-6'),
    (lab_core['LOINC'] == '26474-7') | (lab_core['LOINC'] == '30364-4') | (lab_core['LOINC'] == '731-0') | (lab_core['LOINC'] == '732-8')]

choices = ['creatinine', 
           'hemoglobin', 
           'wbc', 
           'neutrophil_count',  
           'albumin', 
           'total_bilirubin', 
           'sodium', 
           'bicarb',
           'calcium',
           'ast', 
           'alt',
           'platelet',
           'potassium', 
           'chloride',
           'bun',
           'alp',
           'lymphocyte_count']

lab_core.loc[:, 'lab_name'] = np.select(conditions, choices, default="Unknown").astype(str)

In [346]:
lab_core['lab_name'].value_counts()

lab_name
platelet            219831
hemoglobin          214562
wbc                 213181
lymphocyte_count    200662
neutrophil_count    179419
creatinine          176759
potassium           173995
sodium              173523
bicarb              173276
bun                 173135
calcium             172498
chloride            164326
alp                 147214
ast                 146142
alt                 146126
total_bilirubin     145859
albumin             144178
Name: count, dtype: int64

In [347]:
# Remove missing lab values. 
lab_core = lab_core.dropna(subset = ['TestResultCleaned'])

In [348]:
lab_core['TestUnits'].value_counts()

TestUnits
10*3/uL     705744
mg/dL       660061
mmol/L      530919
g/dL        356838
U/L         354286
iU/L         81925
10*3/mm3     16047
10*9/L        6546
cell/uL       3004
/mm3           316
/uL            204
10*3/L         100
10*3/mL         48
g/uL            45
mg/L            39
U/dL             6
mg/mL            4
iU/mL            1
Name: count, dtype: int64

In [349]:
pd.crosstab(lab_core['lab_name'], lab_core['TestUnits'], margins = True, margins_name = "Total")

TestUnits,/mm3,/uL,10*3/L,10*3/mL,10*3/mm3,10*3/uL,10*9/L,U/L,U/dL,cell/uL,g/dL,g/uL,iU/L,iU/mL,mg/L,mg/dL,mg/mL,mmol/L,Total
lab_name,,,,,,,,,,,,,,,,,,,
albumin,0,0,0,0,0,0,0,0,0,0,143788,0,0,0,0,99,0,0,143887
alp,0,0,0,0,0,0,0,118685,6,0,0,0,27880,1,0,0,0,0,146572
alt,0,0,0,0,0,0,0,117737,0,0,0,0,26349,0,0,0,0,0,144086
ast,0,0,0,0,0,0,0,117864,0,0,0,0,27696,0,0,0,0,0,145560
bicarb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,140291,140291
bun,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,172723,0,0,172736
calcium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,170645,4,0,170662
chloride,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,124698,124698
creatinine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,176414,0,0,176427


In [350]:
#For cell counts, need conversions for /mm^3, /uL, 10^3/mL, cell/uL; also need for 10^3/L
#For Hgb, need conversion for g/uL
#For alp, need conversion for iU/mL
#For bun, calcium, creatinine, need conversion for mg/L
conditions = [
    ((lab_core['lab_name'] == 'wbc') | (lab_core['lab_name'] == 'neutrophil_count') | (lab_core['lab_name'] == 'platelet') | (lab_core['lab_name'] == 'lymphocyte_count')) & 
    (lab_core['TestUnits'] == '10*3/L'),
    ((lab_core['lab_name'] == 'wbc') | (lab_core['lab_name'] == 'neutrophil_count') | (lab_core['lab_name'] == 'platelet') | (lab_core['lab_name'] == 'lymphocyte_count')) & 
    ((lab_core['TestUnits'] == '/mm3') | (lab_core['TestUnits'] == '/uL') | (lab_core['TestUnits'] == '10*3/mL')),
    (lab_core['lab_name'] == 'hemoglobin') & (lab_core['TestUnits'] == 'g/uL'),
    (lab_core['lab_name'] == 'alp') & (lab_core['TestUnits'] == 'iU/mL'),
    ((lab_core['lab_name'] == 'bun') | (lab_core['lab_name'] == 'calcium') | (lab_core['lab_name'] == 'creatinine'))  & (lab_core['TestUnits'] == 'mg/L')]

choices = [lab_core['TestResultCleaned'] * 1000000,
           lab_core['TestResultCleaned'] * 1000,
           lab_core['TestResultCleaned'] / 100000,
           lab_core['TestResultCleaned'] * 1000,
           lab_core['TestResultCleaned'] / 10]

lab_core.loc[:, 'test_result_cleaned'] = np.select(conditions, choices, default = lab_core['TestResultCleaned'])

In [351]:
# Elgibliity window is -90 and +30 from advanced diagnosis. 
lab_core['StartDate'] = pd.to_datetime(lab_core['StartDate'], format="%Y-%m-%d")
lab_core_win = (
    lab_core
    .assign(lab_date_diff = (lab_core['ResultDate'] - lab_core['StartDate']).dt.days)
    .query('lab_date_diff >= -90 and lab_date_diff <= 30')
    .filter(items = ['PatientID', 'ResultDate', 'TestResultCleaned', 'lab_name', 'StartDate', 'test_result_cleaned', 'lab_date_diff'])
)

In [352]:
row_ID(lab_core_win)

(462632, 6331)

In [353]:
lab_core_win.loc[:, 'lab_date_diff'] = lab_core_win['lab_date_diff'].abs()

In [354]:
# Select lab closest to date of advanced diagnosis and pivot to a wide table. 
lab_diag_wide = (
    lab_core_win
    .loc[lab_core_win.groupby(['PatientID', 'lab_name'])['lab_date_diff'].idxmin()]
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_diag',
        'creatinine': 'creatinine_diag',
        'hemoglobin': 'hemoglobin_diag',
        'neutrophil_count': 'neutrophil_count_diag',
        'lymphocyte_count': 'lymphocyte_count_diag',
        'total_bilirubin': 'total_bilirubin_diag',
        'wbc': 'wbc_diag',
        'sodium': 'sodium_diag', 
        'bicarb': 'bicarb_diag',
        'calcium': 'calcium_diag',
        'ast': 'ast_diag', 
        'alt': 'alt_diag',
        'platelet': 'platelet_diag',
        'potassium': 'potassium_diag',
        'chloride': 'chloride_diag',
        'bun': 'bun_diag',
        'alp': 'alp_diag'})
)

lab_diag_wide.columns.name = None

In [355]:
row_ID(lab_diag_wide)

(6331, 6331)

In [356]:
lab_diag_wide.sample(3)

,PatientID,albumin_diag,alp_diag,alt_diag,ast_diag,bicarb_diag,bun_diag,calcium_diag,chloride_diag,creatinine_diag,hemoglobin_diag,lymphocyte_count_diag,neutrophil_count_diag,platelet_diag,potassium_diag,sodium_diag,total_bilirubin_diag,wbc_diag
5113,FCF7B8CAAA06E,34.0,71.0,10.0,16.0,21.0,29.0,9.0,NaN,1.60,8.4,1.70,14.2,539.0,NaN,NaN,0.20,18.3
5186,FD23E8661347F,32.0,112.0,27.0,17.0,30.0,14.0,9.3,105.0,1.40,10.6,1.50,NaN,204.0,4.00,142.0,0.38,9.4
4703,FBF17E2BE7BA7,47.0,72.0,6.0,12.0,NaN,13.0,10.1,102.7,0.99,15.2,2.13,7.8,416.0,3.67,139.2,0.90,10.0


In [357]:
#Add back patients without lab data
lab_diag_wide = (
    pd.concat(
        [lab_diag_wide, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(lab_diag_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [358]:
row_ID(lab_diag_wide)

(6461, 6461)

In [359]:
# Create missing variables for labs at time of diagnosis. 
for x in range (1, len(lab_diag_wide.columns)):
    lab_diag_wide.loc[:, lab_diag_wide.columns[x]+'_na'] = np.where(lab_diag_wide[lab_diag_wide.columns[x]].isna(), 1, 0)

In [360]:
list(lab_diag_wide.columns)

['PatientID',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'lymphocyte_count_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_diag',
 'albumin_diag_na',
 'alp_diag_na',
 'alt_diag_na',
 'ast_diag_na',
 'bicarb_diag_na',
 'bun_diag_na',
 'calcium_diag_na',
 'chloride_diag_na',
 'creatinine_diag_na',
 'hemoglobin_diag_na',
 'lymphocyte_count_diag_na',
 'neutrophil_count_diag_na',
 'platelet_diag_na',
 'potassium_diag_na',
 'sodium_diag_na',
 'total_bilirubin_diag_na',
 'wbc_diag_na']

Mean, max, min, and standard deviation

In [361]:
# Elgibility window is negative infinity to +30 from start date. 
lab_core_win_summ = (
    lab_core
    .assign(lab_date_diff = (lab_core['ResultDate'] - lab_core['StartDate']).dt.days)
    .query('lab_date_diff <= 30')
    .filter(items = ['PatientID', 'ResultDate', 'TestResultCleaned', 'lab_name', 'StartDate', 'test_result_cleaned', 'lab_date_diff'])
)

In [362]:
# Pivot table of average values for core labs during elgibility period of negative infinity to + 30 days from start date. 
lab_avg_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].mean()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_avg',
        'creatinine': 'creatinine_avg',
        'hemoglobin': 'hemoglobin_avg',
        'neutrophil_count': 'neutrophil_count_avg',
        'lymphocyte_count': 'lymphocyte_count',
        'total_bilirubin': 'total_bilirubin_avg',
        'wbc': 'wbc_avg',
        'sodium': 'sodium_avg', 
        'bicarb': 'bicarb_avg',
        'calcium': 'calcium_avg',
        'ast': 'ast_avg', 
        'alt': 'alt_avg',
        'platelet': 'platelet_avg',
        'potassium': 'potassium_avg',
        'chloride': 'chloride_avg',
        'bun': 'bun_avg',
        'alp': 'alp_avg'})
)

lab_avg_wide.columns.name = None

In [363]:
row_ID(lab_avg_wide)

(6357, 6357)

In [364]:
# Pivot table of maximum values for core labs during elgibility period of negative infinity to +30 days from start date. 
lab_max_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].max()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_max',
        'creatinine': 'creatinine_max',
        'hemoglobin': 'hemoglobin_max',
        'neutrophil_count': 'neutrophil_count_max',
        'lymphocyte_count': 'lymphocyte_count_max',
        'total_bilirubin': 'total_bilirubin_max',
        'wbc': 'wbc_max', 
        'sodium': 'sodium_max', 
        'bicarb': 'bicarb_max',
        'calcium': 'calcium_max',
        'ast': 'ast_max', 
        'alt': 'alt_max',
        'platelet': 'platelet_max',
        'potassium': 'potassium_max',
        'chloride': 'chloride_max',
        'bun': 'bun_max', 
        'alp': 'alp_max'})
)

lab_max_wide.columns.name = None

In [365]:
row_ID(lab_max_wide)

(6357, 6357)

In [366]:
# Pivot table of minimum values for core labs during elgibility period of negative infinity to +30 days from start date. 
lab_min_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].min()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_min',
        'creatinine': 'creatinine_min',
        'hemoglobin': 'hemoglobin_min',
        'neutrophil_count': 'neutrophil_count_min',
        'lymphocyte_count': 'lymphocyte_count_min',
        'total_bilirubin': 'total_bilirubin_min',
        'wbc': 'wbc_min',
        'sodium': 'sodium_min', 
        'bicarb': 'bicarb_min',
        'calcium': 'calcium_min',
        'ast': 'ast_min', 
        'alt': 'alt_min',
        'platelet': 'platelet_min',
        'potassium': 'potassium_min',
        'chloride': 'chloride_min',
        'bun': 'bun_min',
        'alp': 'alp_min'})
)

lab_min_wide.columns.name = None

In [367]:
row_ID(lab_min_wide)

(6357, 6357)

In [368]:
# Pivot table of standard deviation for core labs during elgibility period of negative infinity to +30 days from start date. 
lab_std_wide = (
    lab_core_win_summ
    .groupby(['PatientID', 'lab_name'])['test_result_cleaned'].std()
    .to_frame()
    .reset_index()
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'test_result_cleaned')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_std',
        'creatinine': 'creatinine_std',
        'hemoglobin': 'hemoglobin_std',
        'neutrophil_count': 'neutrophil_count_std',
        'lymphocyte_count': 'lymphocyte_count_std',
        'total_bilirubin': 'total_bilirubin_std',
        'wbc': 'wbc_std',
        'sodium': 'sodium_std', 
        'bicarb': 'bicarb_std',
        'calcium': 'calcium_std',
        'ast': 'ast_std', 
        'alt': 'alt_std',
        'platelet': 'platelet_std',
        'potassium': 'potassium_std',
        'chloride': 'chloride_std',
        'bun': 'bun_std', 
        'alp': 'alp_std'})
)

lab_std_wide.columns.name = None

In [369]:
lab_summary_wide = pd.merge(lab_avg_wide, lab_max_wide, on = 'PatientID', how = 'outer')

In [370]:
lab_summary_wide = pd.merge(lab_summary_wide, lab_min_wide, on = 'PatientID', how = 'outer')

In [371]:
lab_summary_wide = pd.merge(lab_summary_wide, lab_std_wide, on = 'PatientID', how = 'outer')

In [372]:
row_ID(lab_summary_wide)

(6357, 6357)

In [373]:
#Add back patients without lab data
lab_summary_wide = (
    pd.concat(
        [lab_summary_wide, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(lab_summary_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [374]:
row_ID(lab_summary_wide)

(6461, 6461)

In [375]:
lab_summary_wide.sample(3)

,PatientID,albumin_avg,alp_avg,alt_avg,ast_avg,bicarb_avg,bun_avg,calcium_avg,chloride_avg,creatinine_avg,...,chloride_std,creatinine_std,hemoglobin_std,lymphocyte_count_std,neutrophil_count_std,platelet_std,potassium_std,sodium_std,total_bilirubin_std,wbc_std
1595,F404D061C8B01,41.00,87.000000,9.000000,14.000000,28.000000,13.500000,9.850000,102.0,0.783333,...,2.828427,0.055076,0.565685,0.212132,0.636396,5.196152,0.212132,2.121320,NaN,0.565685
3486,F8E50C7F52CE6,39.00,46.533333,74.533333,51.066667,25.933333,22.133333,9.833333,NaN,1.060000,...,NaN,0.277231,1.485677,0.285804,3.206716,77.900392,NaN,NaN,0.086189,3.343021
4308,FADEF36DCD567,41.25,75.650000,23.050000,18.850000,24.800000,27.400000,9.480000,99.8,1.481500,...,2.667544,0.174183,1.833059,0.435094,1.476752,59.480116,0.318673,2.494204,0.103999,1.668248


In [376]:
list(lab_summary_wide.columns)

['PatientID',
 'albumin_avg',
 'alp_avg',
 'alt_avg',
 'ast_avg',
 'bicarb_avg',
 'bun_avg',
 'calcium_avg',
 'chloride_avg',
 'creatinine_avg',
 'hemoglobin_avg',
 'lymphocyte_count',
 'neutrophil_count_avg',
 'platelet_avg',
 'potassium_avg',
 'sodium_avg',
 'total_bilirubin_avg',
 'wbc_avg',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_max',
 'bicarb_max',
 'bun_max',
 'calcium_max',
 'chloride_max',
 'creatinine_max',
 'hemoglobin_max',
 'lymphocyte_count_max',
 'neutrophil_count_max',
 'platelet_max',
 'potassium_max',
 'sodium_max',
 'total_bilirubin_max',
 'wbc_max',
 'albumin_min',
 'alp_min',
 'alt_min',
 'ast_min',
 'bicarb_min',
 'bun_min',
 'calcium_min',
 'chloride_min',
 'creatinine_min',
 'hemoglobin_min',
 'lymphocyte_count_min',
 'neutrophil_count_min',
 'platelet_min',
 'potassium_min',
 'sodium_min',
 'total_bilirubin_min',
 'wbc_min',
 'albumin_std',
 'alp_std',
 'alt_std',
 'ast_std',
 'bicarb_std',
 'bun_std',
 'calcium_std',
 'chloride_std',
 'creatinine_std',
 

Slope

In [377]:
lab_core_win_summ.loc[:, 'result_date_ordinal'] = lab_core_win_summ['ResultDate'].map(dt.datetime.toordinal)


In [378]:
row_ID(lab_core_win_summ)

(875749, 6357)

In [379]:
#Remove cases where there is only one date for a test
lab_core_win_summ_filtered = lab_core_win_summ[lab_core_win_summ.groupby(['PatientID', 'lab_name'])['result_date_ordinal'].transform('nunique') > 1]

row_ID(lab_core_win_summ_filtered)

(869346, 6163)

In [380]:
lab_slope_wide = (
    lab_core_win_summ_filtered
    .groupby(['PatientID', 'lab_name'])
    .apply(lambda x: pd.Series(linregress(x['result_date_ordinal'], x['test_result_cleaned'])))
    .rename(columns = {0: 'slope'})
    .reset_index()
    .filter(items = ['PatientID', 'lab_name', 'slope'])
    .pivot(index = 'PatientID', columns = 'lab_name', values = 'slope')
    .reset_index()
    .rename(columns = {
        'albumin': 'albumin_slope',
        'creatinine': 'creatinine_slope',
        'hemoglobin': 'hemoglobin_slope',
        'neutrophil_count': 'neutrophil_count_slope',
        'lymphocyte_count': 'lymphocyte_count_slope',
        'total_bilirubin': 'total_bilirubin_slope',
        'wbc': 'wbc_slope',
        'sodium': 'sodium_slope', 
        'bicarb': 'bicarb_slope',
        'calcium': 'calcium_slope',
        'ast': 'ast_slope', 
        'alt': 'alt_slope',
        'platelet': 'platelet_slope',
        'potassium': 'potassium_slope',
        'chloride': 'chloride_slope',
        'bun': 'bun_slope',
        'alp': 'alp_slope'})
)

lab_slope_wide.columns.name = None

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/2057993873.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series(linregress(x['result_date_ordinal'], x['test_result_cleaned'])))


In [381]:
row_ID(lab_slope_wide)

(6163, 6163)

In [382]:
lab_slope_wide = (
    pd.concat(
        [lab_slope_wide, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(lab_slope_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [383]:
row_ID(lab_slope_wide)

(6461, 6461)

In [384]:
# Create missing variables for lab slope. 
for x in range (1, len(lab_slope_wide.columns)):
    lab_slope_wide.loc[:, lab_slope_wide.columns[x]+'_na'] = np.where(lab_slope_wide[lab_slope_wide.columns[x]].isna(), 1, 0)

In [385]:
list(lab_slope_wide.columns)

['PatientID',
 'albumin_slope',
 'alp_slope',
 'alt_slope',
 'ast_slope',
 'bicarb_slope',
 'bun_slope',
 'calcium_slope',
 'chloride_slope',
 'creatinine_slope',
 'hemoglobin_slope',
 'lymphocyte_count_slope',
 'neutrophil_count_slope',
 'platelet_slope',
 'potassium_slope',
 'sodium_slope',
 'total_bilirubin_slope',
 'wbc_slope',
 'albumin_slope_na',
 'alp_slope_na',
 'alt_slope_na',
 'ast_slope_na',
 'bicarb_slope_na',
 'bun_slope_na',
 'calcium_slope_na',
 'chloride_slope_na',
 'creatinine_slope_na',
 'hemoglobin_slope_na',
 'lymphocyte_count_slope_na',
 'neutrophil_count_slope_na',
 'platelet_slope_na',
 'potassium_slope_na',
 'sodium_slope_na',
 'total_bilirubin_slope_na',
 'wbc_slope_na']

Merge

In [386]:
lab_wide = pd.merge(lab_diag_wide, lab_summary_wide, on = 'PatientID')

In [387]:
lab_wide = pd.merge(lab_wide, lab_slope_wide, on = 'PatientID')

In [388]:
row_ID(lab_wide)

(6461, 6461)

In [389]:
list(lab_wide.columns)

['PatientID',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'lymphocyte_count_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_diag',
 'albumin_diag_na',
 'alp_diag_na',
 'alt_diag_na',
 'ast_diag_na',
 'bicarb_diag_na',
 'bun_diag_na',
 'calcium_diag_na',
 'chloride_diag_na',
 'creatinine_diag_na',
 'hemoglobin_diag_na',
 'lymphocyte_count_diag_na',
 'neutrophil_count_diag_na',
 'platelet_diag_na',
 'potassium_diag_na',
 'sodium_diag_na',
 'total_bilirubin_diag_na',
 'wbc_diag_na',
 'albumin_avg',
 'alp_avg',
 'alt_avg',
 'ast_avg',
 'bicarb_avg',
 'bun_avg',
 'calcium_avg',
 'chloride_avg',
 'creatinine_avg',
 'hemoglobin_avg',
 'lymphocyte_count',
 'neutrophil_count_avg',
 'platelet_avg',
 'potassium_avg',
 'sodium_avg',
 'total_bilirubin_avg',
 'wbc_avg',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_m

In [390]:
lab_wide.loc[:, 'neutrophil_lymphocyte_ratio_diag'] = (lab_wide['neutrophil_count_diag']) / (lab_wide['lymphocyte_count_diag'])

In [391]:
list(lab_wide.columns)

['PatientID',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'lymphocyte_count_diag',
 'neutrophil_count_diag',
 'platelet_diag',
 'potassium_diag',
 'sodium_diag',
 'total_bilirubin_diag',
 'wbc_diag',
 'albumin_diag_na',
 'alp_diag_na',
 'alt_diag_na',
 'ast_diag_na',
 'bicarb_diag_na',
 'bun_diag_na',
 'calcium_diag_na',
 'chloride_diag_na',
 'creatinine_diag_na',
 'hemoglobin_diag_na',
 'lymphocyte_count_diag_na',
 'neutrophil_count_diag_na',
 'platelet_diag_na',
 'potassium_diag_na',
 'sodium_diag_na',
 'total_bilirubin_diag_na',
 'wbc_diag_na',
 'albumin_avg',
 'alp_avg',
 'alt_avg',
 'ast_avg',
 'bicarb_avg',
 'bun_avg',
 'calcium_avg',
 'chloride_avg',
 'creatinine_avg',
 'hemoglobin_avg',
 'lymphocyte_count',
 'neutrophil_count_avg',
 'platelet_avg',
 'potassium_avg',
 'sodium_avg',
 'total_bilirubin_avg',
 'wbc_avg',
 'albumin_max',
 'alp_max',
 'alt_max',
 'ast_m

In [392]:
%whos DataFrame

Variable                     Type         Data/Info
---------------------------------------------------
biomarker_wide               DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cohort                       DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics                 DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
ecog_diagnosis_wide          DataFrame                PatientID eco<...>\n[6461 rows x 2 columns]
enhanced_adv                 DataFrame              PatientID diagn<...>n[6461 rows x 16 columns]
lab                          DataFrame                 PatientID   <...>615579 rows x 18 columns]
lab_avg_wide                 DataFrame              PatientID  albu<...>n[6357 rows x 18 columns]
lab_core                     DataFrame                 PatientID Re<...>716133 rows x 11 columns]
lab_core_win                 DataFrame                 PatientID Re<...>[462632 rows x 7 columns]
lab_core_win_s

In [393]:
del lab
del lab_avg_wide
del lab_core
del lab_core_win
del lab_core_win_summ
del lab_core_win_summ_filtered
del lab_diag_wide
del lab_max_wide
del lab_min_wide
del lab_slope_wide
del lab_std_wide
del lab_summary_wide

8. Clean medication dataset

In [394]:
med_admin = pd.read_csv('../data/MedicationAdministration.csv')

In [395]:
med_admin = med_admin[med_admin['PatientID'].isin(cohort_IDs)]

In [396]:
row_ID(med_admin)

(568859, 6341)

In [397]:
med_admin.sample(3)

,PatientID,PracticeID,OrderID,DrugName,CommonDrugName,Route,DrugCategory,DetailedDrugCategory,AdministeredDate,AdministeredAmount,AdministeredUnits
950394,F19BC031D6D72,S291CF564BD92,M6F2B1097A892E1B9FAA11A777D95FC15,gemcitabine hcl,gemcitabine,Intravenous,antineoplastic,chemotherapy,2023-04-25,1540.0,mg
464094,F051AA0FA724E,S507DCE5DD8C6,M3C08400C965F00B409483A6794EA8CAD,dexamethasone,dexamethasone,Oral,steroid,glucocorticoid,2022-05-23,12.0,mg
975503,F2BC9556AE0E2,S6DCB6AF0A70F,M0D8322B3BF23E380483F48F5B60F52D6,granisetron hcl,granisetron,Intravenous,antiemetic,antiemetic,2013-12-02,1.0,mg


In [398]:
med_admin['DrugCategory'].value_counts()

DrugCategory
antineoplastic                  138249
solution-fluid                   95190
antiemetic                       86194
hematological agent              61709
steroid                          61677
pain agent                       50574
anti-infective                   27533
anesthetic                       10647
proton pump inhibitor             7583
antihyperglycemic                 7096
bone therapy agent (bta)          6474
cytoprotective                    5579
antidepressant                    4826
hormone                           2977
gout and hyperurecemia agent       963
immunosuppressive                  415
anticholinergic                    378
sedative agent                     282
hyperglycemic                      245
antidote and reversal agent        212
targeted therapy                    45
antithyroid agent                    8
calcimimetic                         3
Name: count, dtype: int64

Medications are stored in long format. It will be necessary to convert to a wide format. An indicator variable for relevant drug categories will be created. Only medication administration prior to the Start Date will be included. First we will filter for these medications.

In [399]:
med_admin = pd.merge(med_admin, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [400]:
med_admin['StartDate'] = pd.to_datetime(med_admin['StartDate'], format="%Y-%m-%d")
med_admin['AdministeredDate'] = pd.to_datetime(med_admin['AdministeredDate'], format="%Y-%m-%d")

In [401]:
med_admin_win = (
    med_admin
    .assign(med_date_diff = (med_admin['AdministeredDate'] - med_admin['StartDate']).dt.days)
    .query('med_date_diff < 0 and med_date_diff >= -90')
)

In [402]:
row_ID(med_admin_win)

(32338, 1347)

In [403]:
med_admin_win['DrugCategory'].value_counts()

DrugCategory
pain agent                      8485
solution-fluid                  6231
hematological agent             3760
anti-infective                  3677
anesthetic                      2100
antiemetic                      1971
proton pump inhibitor           1224
steroid                         1100
antineoplastic                   989
antihyperglycemic                902
antidepressant                   634
hormone                          513
cytoprotective                   419
bone therapy agent (bta)         143
gout and hyperurecemia agent      57
immunosuppressive                 44
hyperglycemic                     34
sedative agent                    31
antidote and reversal agent       18
calcimimetic                       3
anticholinergic                    2
antithyroid agent                  1
Name: count, dtype: int64

SolutionFluid: no indicator variable created

Pain

Opioid PO

In [404]:
# List of avialable opioids in the US. 
opioid_list = [
    'buprenorphine',
    'codeine',
    'fentanyl',
    'hydrocodone',
    'hydromorphone',
    'methadone',
    'morphine',
    'oxycodone',
    'oxymorphone',
    'tapentadol',
    'tramadol'
]

In [405]:
med_admin_win.loc[:, 'opioid_PO_diag'] = (
    np.where(((med_admin_win['Route'] == 'Oral') | 
              (med_admin_win['Route'] == 'Transdermal') | 
              (med_admin_win['Route'] == 'Sublingual')) &
             (med_admin_win['CommonDrugName'].str.contains('|'.join(opioid_list))), 1, 0)
)

Nonopioid PO

In [406]:
med_admin_win.loc[:, 'nonopioid_PO_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'pain agent') & 
             (med_admin_win['Route'] == 'Oral') & 
             (~med_admin_win['CommonDrugName'].str.contains('|'.join(opioid_list))), 1, 0)
)

Pain IV

In [407]:
med_admin_win.loc[:, 'pain_IV_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'pain agent') & 
             (med_admin_win['Route'] == 'Intravenous'), 1, 0)
)

Antineoplastic: no indicator variable created

Hematological agent:

Heparin and other parenteral agents

In [408]:
med_admin_win.loc[:, 'heparin_diag'] = (
    np.where(((med_admin_win['CommonDrugName'].str.contains('heparin')) & 
              (med_admin_win['AdministeredUnits'] == 'unit/kg/hr')) | 
             (med_admin_win['CommonDrugName'].str.contains('bivalirudin')) | 
             (med_admin_win['CommonDrugName'].str.contains('argatroban')), 1, 0)
)

Enoxaparin and other subcutaneous agents

In [409]:
med_admin_win.loc[:, 'enoxaparin_diag'] = (
    np.where(((med_admin_win['CommonDrugName'].str.contains('enoxaparin')) & 
              (med_admin_win['AdministeredAmount'] > 40)) | 
             ((med_admin_win['CommonDrugName'].str.contains('dalteparin')) & 
              (med_admin_win['AdministeredAmount'] > 5000)) | 
             ((med_admin_win['CommonDrugName'].str.contains('fondaparinux')) & 
              (med_admin_win['AdministeredAmount'] > 2.5)), 1, 0)
)

DOAC

In [410]:
med_admin_win.loc[:, 'doac_diag'] = (
    np.where((med_admin_win['CommonDrugName'].str.contains('apixaban')) | 
             (med_admin_win['CommonDrugName'].str.contains('rivaroxaban')) | 
             (med_admin_win['CommonDrugName'].str.contains('dabigatran')) | 
             (med_admin_win['CommonDrugName'].str.contains('edoxaban')), 1, 0)
)

Warfarin

In [411]:
med_admin_win.loc[:, 'warfarin_diag'] = np.where((med_admin_win['CommonDrugName'].str.contains('warfarin')), 1, 0)

Anticoagulation merge

In [412]:
med_admin_win['ac_diag'] = np.where(
    (med_admin_win['heparin_diag'] + med_admin_win['enoxaparin_diag'] + med_admin_win['doac_diag'] + med_admin_win['warfarin_diag']) > 0,
    1,
    0
)

In [413]:
# Drop heparin, enoxaparin, DOAC, and warfarin columns. 
med_admin_win = med_admin_win.drop(columns = ['heparin_diag', 'enoxaparin_diag', 'doac_diag', 'warfarin_diag'])

Antiemetic: no indicator variable created

Anti-infective:

Anti-infective IV

In [414]:
med_admin_win.loc[:, 'antiinfective_IV_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'anti-infective') & 
             (med_admin_win['Route'] == 'Intravenous'), 1, 0)
)

Anti-infective PO

In [415]:
med_admin_win.loc[:, 'antiinfective_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'anti-infective') & 
             (med_admin_win['Route'] == 'Oral'), 1, 0)
)

Steroid:

In [416]:
med_admin_win.loc[:, 'steroid_diag'] = (
    np.where((med_admin_win['DrugCategory'] == 'steroid') & 
             ((med_admin_win['Route'] == 'Intravenous') | 
              (med_admin_win['Route'] == 'Oral') | 
              (med_admin_win['Route'] == 'Intrajejunal') |
              (med_admin_win['Route'] == 'Nasogastric') |
              (med_admin_win['Route'] == 'enteral')), 1, 0)
)

Anesthetic: no indicator variable created

Antihyperglycemic: 

In [417]:
med_admin_win.loc[:, 'antihyperglycemic_diag'] = np.where(med_admin_win['DrugCategory'] == 'antihyperglycemic', 1, 0)

Proton pump inhibitor:

In [418]:
med_admin_win.loc[:, 'ppi_diag'] = np.where(med_admin_win['DrugCategory'] == 'proton pump inhibitor', 1, 0)

Antidepressant:

In [419]:
med_admin_win.loc[:, 'antidepressant_diag'] = np.where(med_admin_win['DrugCategory'] == 'antidepressant', 1, 0)

Cytoprotective: no indicator variable created

Hormone:

In [420]:
med_admin_win.query('DrugCategory == "hormone"')['CommonDrugName'].value_counts()

CommonDrugName
levothyroxine            418
megestrol                 37
liothyronine              13
octreotide                13
calcitonin                12
vasopressin               11
testosterone               4
lanreotide                 2
estrogens, conjugated      2
estradiol                  1
Name: count, dtype: int64

In [421]:
med_admin_win.loc[:, 'thyroid_diag'] = np.where(med_admin_win['CommonDrugName'] == 'levothyroxine', 1, 0)

Gout and hyperuricemic agent: no indcator variable created

Bone therapy agent:

In [422]:
med_admin_win.loc[:, 'bta_diag'] = np.where(med_admin_win['DrugCategory'] == 'bone therapy agent (bta)', 1, 0)

Immunosuppressive:

In [423]:
med_admin_win.query('DrugCategory == "immunosuppressive"')['CommonDrugName'].value_counts()

CommonDrugName
cyclosporine         22
tacrolimus           10
mycophenolic acid     9
azathioprine          3
Name: count, dtype: int64

In [424]:
med_admin_win.loc[:, 'is_diag'] = np.where(med_admin_win['DrugCategory'] == 'immunosuppressive', 1, 0)

Antidote and reversal agent: no indicator variable created

Sedative agent: no indicator variable created

Hyperglycemic: no indicator variable created

Anticholinergic: no indicator variable created

Antithyroid agent: no indicator variable created

Calcimemetic: no indicator variable created

Targeted therapy: no indicator variable created

Condense columns and rows:

In [425]:
# Select columns with indicator variables and PatientID, then collapse rows by PatientID and sum columns. 
med_admin_wide = (
    med_admin_win
    [med_admin_win.columns[med_admin_win.columns.str.contains('diag|PatientID')]]
    .groupby('PatientID').sum()
)

In [426]:
med_admin_wide.sample(3)

,opioid_PO_diag,nonopioid_PO_diag,pain_IV_diag,ac_diag,antiinfective_IV_diag,antiinfective_diag,steroid_diag,antihyperglycemic_diag,ppi_diag,antidepressant_diag,thyroid_diag,bta_diag,is_diag
PatientID,,,,,,,,,,,,,
F9958D282F4DC,0,0,0,0,0,0,0,0,0,0,0,0,0
F30F0FC1576AD,0,0,0,0,0,0,0,0,0,0,0,0,0
F2386AD804E02,0,0,0,0,0,0,0,0,0,0,0,0,0


In [427]:
# Replace numbers greater than 1 with 1; 0 remains unchanged. 
med_admin_wide = (
    med_admin_wide.mask(med_admin_wide > 1, 1)
    .reset_index()
)

In [428]:
row_ID(med_admin_wide)

(1347, 1347)

In [429]:
# Append missing training IDs
med_admin_wide = (
    pd.concat(
        [med_admin_wide, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(med_admin_wide['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [430]:
row_ID(med_admin_wide)

(6461, 6461)

In [431]:
med_admin_wide.sample(3)

,PatientID,opioid_PO_diag,nonopioid_PO_diag,pain_IV_diag,ac_diag,antiinfective_IV_diag,antiinfective_diag,steroid_diag,antihyperglycemic_diag,ppi_diag,antidepressant_diag,thyroid_diag,bta_diag,is_diag
871,F7731FB392AB9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2341,F4E6E0AD5B480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1154,F92FD54F1E818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [432]:
med_admin_wide = med_admin_wide.fillna(0)

In [433]:
med_admin_wide.sample(3)

,PatientID,opioid_PO_diag,nonopioid_PO_diag,pain_IV_diag,ac_diag,antiinfective_IV_diag,antiinfective_diag,steroid_diag,antihyperglycemic_diag,ppi_diag,antidepressant_diag,thyroid_diag,bta_diag,is_diag
2065,F46FB1CE6DE50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
601,F72D09624ACE4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4672,F5F3E9B349AF4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [434]:
# Percent of patients receiving relevant medications at time of metastatic diagnosis. 
(med_admin_wide.iloc[:, 1:].sum()/len(med_admin_wide)).sort_values(ascending = False)

pain_IV_diag              0.070887
antiinfective_IV_diag     0.069494
steroid_diag              0.055100
opioid_PO_diag            0.044420
nonopioid_PO_diag         0.042099
antiinfective_diag        0.025228
ppi_diag                  0.022907
bta_diag                  0.018728
antihyperglycemic_diag    0.012227
antidepressant_diag       0.011608
ac_diag                   0.010215
thyroid_diag              0.007584
is_diag                   0.000929
dtype: float64

In [435]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cohort                DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics          DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6461 rows x 2 columns]
enhanced_adv          DataFrame              PatientID diagn<...>n[6461 rows x 16 columns]
lab_wide              DataFrame              PatientID  albu<...>[6461 rows x 138 columns]
med_admin             DataFrame                PatientID    <...>568859 rows x 12 columns]
med_admin_wide        DataFrame              PatientID  opio<...>n[6461 rows x 14 columns]
med_admin_win         DataFrame                PatientID    <...>[32338 rows x 26 columns]
vitals_wide           DataFrame              PatientID sbp_d<...>\n[6461 rows x 6 columns]


In [436]:
del med_admin
del med_admin_win

9. Clean Diagnosis dataset

In [437]:
diagnosis = pd.read_csv('../data/Diagnosis.csv')

In [438]:
diagnosis = diagnosis[diagnosis['PatientID'].isin(cohort_IDs)]

In [439]:
row_ID(diagnosis)

(309101, 6461)

Elixhauser

In [440]:
diagnosis = pd.merge(diagnosis, cohort[['PatientID', 'StartDate']], on = 'PatientID', how = 'left')

In [443]:
diagnosis['StartDate'] = pd.to_datetime(diagnosis['StartDate'], format="%Y-%m-%d")
diagnosis['DiagnosisDate'] = pd.to_datetime(diagnosis['DiagnosisDate'], format="%Y-%m-%d")

In [444]:
diagnosis.loc[:, 'diagnosis_date_diff'] = (diagnosis['DiagnosisDate'] - diagnosis['StartDate']).dt.days

In [445]:
# Remove decimal to make mapping to Elixhauser easier. 
diagnosis.loc[:, 'diagnosis_code'] = diagnosis['DiagnosisCode'].replace('\.', '', regex = True)

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/3887710887.py:2: SyntaxWarning: invalid escape sequence '\.'
  diagnosis.loc[:, 'diagnosis_code'] = diagnosis['DiagnosisCode'].replace('\.', '', regex = True)


Elixhauser for ICD-9

In [446]:
# ICD-9 dataframe with unique codes for each patient. 
diagnosis_elix_9 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-9-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [447]:
row_ID(diagnosis_elix_9)

(17925, 2392)

In [448]:
diagnosis_elix_9.loc[:, 'chf'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('39891|'
                                                          '402(01|11|91)|'
                                                          '404(01|03|[19][13])|'
                                                          '42(5[456789]|8)'), 1, 0)
)

In [449]:
diagnosis_elix_9.loc[:, 'cardiac_arrhythmias'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('426([079]|1[023])|'
                                                          '427[012346789]|'
                                                          '7850|'
                                                          '996(01|04)|'
                                                          'V450|'
                                                          'V533'), 1, 0)
)

In [450]:
diagnosis_elix_9.loc[:, 'valvular_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('0932|'
                                                          '39[4567]|'
                                                          '424|'
                                                          '746[3456]|'
                                                          'V422|'
                                                          'V433'), 1, 0)
)

In [451]:
diagnosis_elix_9.loc[:, 'pulmonary_circulation'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('41(5[01]|6|7[089])'), 1, 0)
)

In [452]:
diagnosis_elix_9.loc[:, 'peripheral_vascular'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('0930|'
                                                          '4373|'
                                                          '44([01]|3[123456789]|71)|'
                                                          '557[19]|'
                                                          'V434'), 1, 0)
)

In [453]:
diagnosis_elix_9.loc[:, 'htn_uncomplicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('401'), 1, 0)
)

In [454]:
diagnosis_elix_9.loc[:, 'htn_complicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('40[2345]'), 1, 0)
)

In [455]:
diagnosis_elix_9.loc[:, 'paralysis'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('3341|'
                                                          '34([23]|4[01234569])'), 1, 0)
)

In [456]:
diagnosis_elix_9.loc[:, 'other_neuro_disorders'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('33(19|2[01]|3([45]|92)|[45]|62)|'
                                                          '34([015]|8[13])|'
                                                          '78[04]3'), 1, 0)
)

In [457]:
diagnosis_elix_9.loc[:, 'chronic_pulmonary'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('416[89]|'
                                                          '49|'
                                                          '50([012345]|64|8[18])'), 1, 0)
)

In [458]:
diagnosis_elix_9.loc[:, 'diabetes_uncomplicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('250[0123]'), 1, 0)
)

In [459]:
diagnosis_elix_9.loc[:, 'diabetes_complicated'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('250[456789]'), 1, 0)
)

In [460]:
diagnosis_elix_9.loc[:, 'hypothyroidism'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2409|'
                                                          '24([34]|6[18])'), 1, 0)
)

In [461]:
diagnosis_elix_9.loc[:, 'renal_failure'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('403[019]1|'
                                                          '404[019][23]|'
                                                          '58([56]|80)|'
                                                          'V4(20|51)|'
                                                          'V56'), 1, 0)
)

In [462]:
diagnosis_elix_9.loc[:, 'liver_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('070(2[23]]|3[23]|44|54|6|9)|'
                                                          '456[012]|'
                                                          '57([01]|2[2345678]|3[3489])|'
                                                          'V427'), 1, 0)
)

In [463]:
diagnosis_elix_9.loc[:, 'peptic_ulcer_disease'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('53[1234][79]'), 1, 0)
)

In [464]:
diagnosis_elix_9.loc[:, 'aids_hiv'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('04[234]'), 1, 0)
)

In [465]:
diagnosis_elix_9.loc[:, 'lymphoma'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('20([012]|30)|'
                                                          '2386'), 1, 0)
)

In [466]:
diagnosis_elix_9.loc[:, 'metastatic_cancer'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('19[6789]'), 1, 0)
)

In [467]:
diagnosis_elix_9.loc[:, 'solid_tumor_wout_mets'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('1[456]|'
                                                          '17[012456789]|'
                                                          '18|'
                                                          '19([012345])'), 1, 0)
)

In [468]:
diagnosis_elix_9.loc[:, 'rheumatoid_arthritis'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('446|'
                                                          '7010|'
                                                          '71(0[0123489]|12|4|93)|'
                                                          '72([05]|85|889|930)'), 1, 0)
)

In [469]:
diagnosis_elix_9.loc[:, 'coagulopathy'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('28(6|7[1345])'), 1, 0)
)

In [470]:
diagnosis_elix_9.loc[:, 'obesity'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2780'), 1, 0)
)

In [471]:
diagnosis_elix_9.loc[:, 'weight_loss'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('26[0123]|'
                                                          '7832|'
                                                          '7994'), 1, 0)
)

In [472]:
diagnosis_elix_9.loc[:, 'fluid_electrolyte'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2(536|76)'), 1, 0)
)

In [473]:
diagnosis_elix_9.loc[:, 'blood_loss_anemia'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2800'), 1, 0)
)

In [474]:
diagnosis_elix_9.loc[:, 'deficiency_anemia'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('28(0[123456789]|1)'), 1, 0)
)

In [475]:
diagnosis_elix_9.loc[:, 'alcohol_abuse'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2652|'
                                                          '291[12356789]|'
                                                          '30(3[09]|50)|'
                                                          '3575|'
                                                          '4255|'
                                                          '5353|'
                                                          '571[0123]|'
                                                          '980|'
                                                          'V113'), 1, 0)
)

In [476]:
diagnosis_elix_9.loc[:, 'drug_abuse'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('292|'
                                                          '30(4|5[23456789])|'
                                                          'V6542'), 1, 0)
)

In [477]:
diagnosis_elix_9.loc[:, 'psychoses'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('2938|'
                                                          '296[0145]4|'
                                                          '29[578]'), 1, 0)
)

In [478]:
diagnosis_elix_9.loc[:, 'depression'] = (
    np.where(diagnosis_elix_9['diagnosis_code'].str.match('296[235]|'
                                                          '3(004|09|11)'), 1, 0)
)

In [480]:
# Create variable that captures ICD-9 codes not included in Elixhauser. 
diagnosis_elix_9.loc[:, 'elixhauser_other'] = (
    np.where(diagnosis_elix_9.iloc[:, 3:].eq(0).all(1), 1, 0)
)

In [481]:
# Single-row-per-patient dataframe with columns as Elixhauser comorbidities. 
diagnosis_elix_9_wide = (
    diagnosis_elix_9
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [482]:
row_ID(diagnosis_elix_9_wide)

(2392, 2392)

Elixhauser for ICD-10

In [483]:
# ICD-10 dataframe with unique codes for each patient.  
diagnosis_elix_10 = (
    diagnosis
    .query('diagnosis_date_diff <= 30')
    .query('DiagnosisCodeSystem == "ICD-10-CM"')
    .drop_duplicates(subset = (['PatientID', 'DiagnosisCode']), keep = 'first')
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [484]:
row_ID(diagnosis_elix_10)

(63437, 5199)

In [485]:
diagnosis_elix_10.loc[:, 'chf'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I099|'
                                                           'I1(10|3[02])|'
                                                           'I255|'
                                                           'I4(2[056789]|3)|'
                                                           'I50|'
                                                           'P290'), 1, 0)
)

In [486]:
diagnosis_elix_10.loc[:, 'cardiac_arrhythmias'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I4(4[123]|5[69]|[789])|'
                                                           'R00[018]|'
                                                           'T821|'
                                                           'Z[49]50'), 1, 0)
)

In [487]:
diagnosis_elix_10.loc[:, 'valvular_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('A520|'
                                                           'I0([5678]|9[18])|'
                                                           'I3[456789]|'
                                                           'Q23[0123]|'
                                                           'Z95[234]'), 1, 0)
)

In [488]:
diagnosis_elix_10.loc[:, 'pulmonary_circulation'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I2([67]|8[089])'), 1, 0)
)

In [489]:
diagnosis_elix_10.loc[:, 'peripheral_vascular'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I7([01]|3[189]|71|9[02])|'
                                                           'K55[189]|'
                                                           'Z95[89]'), 1, 0)
)

In [490]:
diagnosis_elix_10.loc[:, 'htn_uncomplicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I10'), 1, 0)
)

In [491]:
diagnosis_elix_10.loc[:, 'htn_complicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I1[1235]'), 1, 0)
)

In [492]:
diagnosis_elix_10.loc[:, 'paralysis'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('G041|'
                                                           'G114|'
                                                           'G8(0[12]|[12]|3[012349])'), 1, 0)
)

In [493]:
diagnosis_elix_10.loc[:, 'other_neuro_disorders'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('G1[0123]|'
                                                           'G2([012]|5[45])|'
                                                           'G3(1[289]|[2567])|'
                                                           'G4[01]|'
                                                           'G93[14]|'
                                                           'R470|'
                                                           'R56'), 1, 0)
)

In [494]:
diagnosis_elix_10.loc[:, 'chronic_pulmonary'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I27[89]|'
                                                           'J4[01234567]|'
                                                           'J6([01234567]|84)|'
                                                           'J70[13]'), 1, 0)
)

In [495]:
diagnosis_elix_10.loc[:, 'diabetes_uncomplicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E1[01234][019]'), 1, 0)
)

In [496]:
diagnosis_elix_10.loc[:, 'diabetes_complicated'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E1[01234][2345678]'), 1, 0)
)

In [497]:
diagnosis_elix_10.loc[:, 'hypothyroidism'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E0[0123]|'
                                                           'E890'), 1, 0)
)

In [498]:
diagnosis_elix_10.loc[:, 'renal_failure'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('I1(20|31)|'
                                                           'N1[89]|'
                                                           'N250|'
                                                           'Z49[012]|'
                                                           'Z9(40|92)'), 1, 0)
)

In [499]:
diagnosis_elix_10.loc[:, 'liver_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('B18|'
                                                           'I8(5|64)|'
                                                           'I982|'
                                                           'K7(0|1[13457]|[234]|6[023456789])|'
                                                           'Z944'), 1, 0)
)

In [500]:
diagnosis_elix_10.loc[:, 'peptic_ulcer_disease'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('K2[5678][79]'), 1, 0)
)

In [501]:
diagnosis_elix_10.loc[:, 'aids_hiv'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('B2[0124]'), 1, 0)
)

In [502]:
diagnosis_elix_10.loc[:, 'lymphoma'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C8[123458]|'
                                                           'C9(0[02]|6)'), 1, 0)
)

In [503]:
diagnosis_elix_10.loc[:, 'metastatic_cancer'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C(7[789]|80)'), 1, 0)
)

In [504]:
diagnosis_elix_10.loc[:, 'solid_tumor_wout_mets'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('C[01]|'
                                                           'C2[0123456]|'
                                                           'C3[01234789]|'
                                                           'C4[01356789]|'
                                                           'C5[012345678]|'
                                                           'C6|'
                                                           'C7[0123456]|'
                                                           'C97'), 1, 0)
)

In [505]:
diagnosis_elix_10.loc[:, 'rheumatoid_arthritis'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('L94[013]|'
                                                           'M0[568]|'
                                                           'M12[03]|'
                                                           'M3(0|1[0123]|[2345])|'
                                                           'M4(5|6[189])'), 1, 0)
)

In [506]:
diagnosis_elix_10.loc[:, 'coagulopathy'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D6([5678]|9[13456])'), 1, 0)
)

In [507]:
diagnosis_elix_10.loc[:, 'obesity'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E66'), 1, 0)
)

In [508]:
diagnosis_elix_10.loc[:, 'weight_loss'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E4[0123456]|'
                                                           'R6(34|4)'), 1, 0)
)

In [509]:
diagnosis_elix_10.loc[:, 'fluid_electrolyte'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('E222|'
                                                           'E8[67]'), 1, 0)
)

In [510]:
diagnosis_elix_10.loc[:, 'blood_loss_anemia'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D500'), 1, 0)
)

In [511]:
diagnosis_elix_10.loc[:, 'deficiency_anemia'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('D5(0[89]|[123])'), 1, 0)
)

In [512]:
diagnosis_elix_10.loc[:, 'alcohol_abuse'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F10|'
                                                           'E52|'
                                                           'G621|'
                                                           'I426|'
                                                           'K292|'
                                                           'K70[039]|'
                                                           'T51|'
                                                           'Z502|'
                                                           'Z7(14|21)'), 1, 0)
)

In [513]:
diagnosis_elix_10.loc[:, 'drug_abuse'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F1[12345689]|'
                                                           'Z7(15|22)'), 1, 0)
)

In [514]:
diagnosis_elix_10.loc[:, 'psychoses'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F2[0234589]|'
                                                           'F3([01]2|15)'), 1, 0)
)

In [515]:
diagnosis_elix_10.loc[:, 'depression'] = (
    np.where(diagnosis_elix_10['diagnosis_code'].str.match('F204|'
                                                           'F3(1[345]|[23]|41)|'
                                                           'F4[13]2'), 1, 0)
)

In [517]:
# Create variable that captures ICD-10 codes not included in Elixhauser. 
diagnosis_elix_10.loc[:, 'elixhauser_other'] = (
    np.where(diagnosis_elix_10.iloc[:, 3:].eq(0).all(1), 1, 0)
)

In [518]:
diagnosis_elix_10_wide = (
    diagnosis_elix_10
    .drop(columns = ['DiagnosisCode', 'diagnosis_code'])
    .groupby('PatientID').sum()
    .reset_index()
)

In [519]:
row_ID(diagnosis_elix_10_wide)

(5199, 5199)

In [520]:
# Merge Elixhauser 9 and 10 and sum by PatientID.
diagnosis_elixhauser = (
    pd.concat([diagnosis_elix_9_wide, diagnosis_elix_10_wide])
    .groupby('PatientID').sum()
)

In [521]:
# Create unqiue ICD count for each patient. 
diagnosis_elixhauser['icd_count'] = diagnosis_elixhauser.sum(axis = 1)

In [523]:
# Other than unique ICD count, values greater than 1 are set to 1; 0 remains unchanged. 
diagnosis_elixhauser.iloc[:, :-1] = (
    diagnosis_elixhauser.iloc[:, :-1].mask(diagnosis_elixhauser.iloc[:, :-1] >1, 1)
)

In [524]:
#Also create calculation of elixhauser index (i.e. without other ICD codes)
diagnosis_elixhauser.sample(3)

,chf,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,htn_uncomplicated,htn_complicated,paralysis,other_neuro_disorders,chronic_pulmonary,...,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemia,alcohol_abuse,drug_abuse,psychoses,depression,elixhauser_other,icd_count
PatientID,,,,,,,,,,,,,,,,,,,,,
F02F1E588031B,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,1,8
F5F80D13E2405,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
FEB56442F933F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6


In [525]:
diagnosis_elixhauser['elixhauser_index'] = diagnosis_elixhauser.loc[
    :, ~diagnosis_elixhauser.columns.isin(['elixhauser_other', 'icd_count'])
].sum(axis=1)

In [526]:
diagnosis_elixhauser.sample(3)

,chf,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,htn_uncomplicated,htn_complicated,paralysis,other_neuro_disorders,chronic_pulmonary,...,fluid_electrolyte,blood_loss_anemia,deficiency_anemia,alcohol_abuse,drug_abuse,psychoses,depression,elixhauser_other,icd_count,elixhauser_index
PatientID,,,,,,,,,,,,,,,,,,,,,
FF6376FA5E14B,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,6,3
FABDAA40A067A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,6,4
F890BFB7C605B,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,9,3


In [527]:
diagnosis_elixhauser = diagnosis_elixhauser.reset_index()

In [528]:
row_ID(diagnosis_elixhauser)

(6455, 6455)

In [529]:
# Append missing cohort IDs
diagnosis_elixhauser = (
    pd.concat(
        [diagnosis_elixhauser, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(diagnosis_elixhauser['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
    .fillna(0)
)

In [530]:
row_ID(diagnosis_elixhauser)

(6461, 6461)

Other Cancer
ICD-9 Cancer Cancer Codes

In [550]:
# Select all ICD-9 cancer codes between 140-209.
# Exclude benign neoplasms: 210-229, carcinoma in site: 230-234, and neoplasms of uncertain behavior or nature: 235-239.
cancer_9 = (
    diagnosis_elix_9[diagnosis_elix_9['DiagnosisCode'].str.startswith(
        ('14','15', '16', '17', '18', '19', '20'))]
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

Remove the following ICD-9 codes representing bladder cancer, metastasis, ill-defined neoplasms, and benign neoplasms of skin (BCC and SCC):

188 - Malignant neoplasm of bladder
187.2 - Malignant neoplasm of bladder
187.3 - Malignant neoplasm of other and unspecified sites of urinary organs
187.5 - Malignant neoplasm of ureter
187.6 - Malignant neoplasm of urethra
187.7 - Malignant neoplasm of other and unspecified sites of genitourinary organs
187.8 - Malignant neoplasm of unspecified site of genitourinary organs
189.1 - Malignant neoplasm of renal pelvis
189.2 - Malignant neoplasm of ureter
189.3 - Malignant neoplasm of urethra
189.8 - Malignant neoplasm of other and unspecified urinary organs
189.9 - Malignant neoplasm of urinary organs, site unspecified
173 - Other and unspecified malignant neoplasm of skin
196 - Secondary and unspecified malignant neoplasm of lymph nodes
197 - Secondary malignant neoplasm of respiratory and digestive systems
198 - Secondary malignant neoplasm of other specified sites
199 - Malignant neoplasm without specification of site

In [597]:
# Dataframe of ICD-9 neoplasm codes that exclude bladder cancer, metastasis, or benign neoplasms.
other_cancer_9 = (
    cancer_9[~cancer_9['diagnosis_code'].str.startswith(('188', '1872', '1873', '1875', '1876', '1877', '1878', '1891', '1892', '1893', '1898', '1899', '173', '196', '197', '198', '199'))]
)

In [598]:
other_cancer_9.loc[:,'other_cancer_9'] = 1

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/4262767015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_cancer_9.loc[:,'other_cancer_9'] = 1


In [599]:
other_cancer_9 = (
    other_cancer_9
    .drop_duplicates(subset = 'PatientID', keep = 'first')
    .filter(items = ['PatientID', 'other_cancer_9'])
)

In [600]:
row_ID(other_cancer_9)

(545, 545)

In [601]:
# Append missing cohort IDs
other_cancer_9 = (
    pd.concat(
        [other_cancer_9, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(other_cancer_9['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
    .fillna(0)
)

In [602]:
row_ID(other_cancer_9)

(6461, 6461)

ICD-10 Cancer codes

In [603]:
# Select all ICD-10 codes between C00-D49 
# Exclude in situ neoplasms: D00-D09, benign neoplasms: D10-D36, benign neuroendocrine tumor: D3A, and neoplasms of unspecified behavior: D37 and D49
cancer_10 = (
    diagnosis_elix_10[diagnosis_elix_10['DiagnosisCode'].str.startswith(
        ('C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D38', 'D39', 'D4'))]
    .filter(items = ['PatientID', 'DiagnosisCode', 'diagnosis_code'])
)

In [604]:
row_ID(cancer_10)

(11885, 5042)

Remove the following ICD-10 codes which capture bladder cancer, metastasis, and benign skin neoplasms(eg., BCC and SCC).

C67 - Malignant neoplasm of bladder
C65 - Malignant neoplasm of renal pelvis
C66 - Malignant neoplasm of the ureter
C68 - Malignant neoplasms of other and unspecified urinary organs (including the urethra, paraurethral glands, overlapping sites, and unspecified urinary organs
C44 - Other and unspecified malignant neoplasm of skin
C77 - Secondary and unspecified malignant neoplasm of lymph nodes
C78 - Secondary malignant neoplasm of respiratory and digestive organs
C79 - Secondary malignant neoplasm of other and unspecified sites
C80 - Malignant neoplasm without specification of site
D47.2 - Monoclonal gammopathy
D48 - Neoplasm of uncertain behavior of other and unspecified sites
D49 - Neoplasms of unspecified behavior

In [605]:
# Dataframe of ICD-10 neoplasm codes that exclude lung cancer, metastasis, or benign neoplasms.
other_cancer_10 = (
    cancer_10[~cancer_10['diagnosis_code'].str.startswith(('C67', 'C65', 'C66', 'C68', 'C44', 'C77', 'C78', 'C79', 'C80', 'D472', 'D48', 'D49'))]
)

In [606]:
other_cancer_10.loc[:,'other_cancer_10'] = 1

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/3927732622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_cancer_10.loc[:,'other_cancer_10'] = 1


In [607]:
# Drop duplicates.
other_cancer_10 = (
    other_cancer_10
    .drop_duplicates(subset = 'PatientID', keep = 'first')
    .filter(items = ['PatientID', 'other_cancer_10'])
)

In [608]:
row_ID(other_cancer_10)

(939, 939)

In [609]:
# Append missing cohort IDs
other_cancer_10 = (
    pd.concat(
        [other_cancer_10, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(other_cancer_10['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
    .fillna(0)
)

In [610]:
row_ID(other_cancer_10)

(6461, 6461)

In [611]:
other_cancer = pd.merge(other_cancer_9, other_cancer_10, on = 'PatientID')

In [612]:
# Combine other_cancer_9 and other_cancer_19; replace values equal to 2 with 1. 
other_cancer = (
    other_cancer
    .assign(other_cancer = other_cancer['other_cancer_9'] + other_cancer['other_cancer_10'])
    .filter(items = ['PatientID', 'other_cancer'])
    .replace(2, 1)
)

In [613]:
row_ID(other_cancer)

(6461, 6461)

In [614]:
other_cancer['other_cancer'].value_counts()

other_cancer
0.0    5089
1.0    1372
Name: count, dtype: int64

Merge:

In [615]:
diagnosis_wide = pd.merge(diagnosis_elixhauser, other_cancer, on = 'PatientID')

In [616]:
diagnosis_wide.sample(3)

,PatientID,chf,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,htn_uncomplicated,htn_complicated,paralysis,other_neuro_disorders,...,blood_loss_anemia,deficiency_anemia,alcohol_abuse,drug_abuse,psychoses,depression,elixhauser_other,icd_count,elixhauser_index,other_cancer
3170,F7F18E19AE7A9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0
1279,F32E5E57E6FE6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0
6354,FFBC845176B9A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0


In [617]:
row_ID(diagnosis_wide)

(6461, 6461)

In [618]:
list(diagnosis_wide.columns)

['PatientID',
 'chf',
 'cardiac_arrhythmias',
 'valvular_disease',
 'pulmonary_circulation',
 'peripheral_vascular',
 'htn_uncomplicated',
 'htn_complicated',
 'paralysis',
 'other_neuro_disorders',
 'chronic_pulmonary',
 'diabetes_uncomplicated',
 'diabetes_complicated',
 'hypothyroidism',
 'renal_failure',
 'liver_disease',
 'peptic_ulcer_disease',
 'aids_hiv',
 'lymphoma',
 'metastatic_cancer',
 'solid_tumor_wout_mets',
 'rheumatoid_arthritis',
 'coagulopathy',
 'obesity',
 'weight_loss',
 'fluid_electrolyte',
 'blood_loss_anemia',
 'deficiency_anemia',
 'alcohol_abuse',
 'drug_abuse',
 'psychoses',
 'depression',
 'elixhauser_other',
 'icd_count',
 'elixhauser_index',
 'other_cancer']

In [619]:
%whos DataFrame

Variable                 Type         Data/Info
-----------------------------------------------
biomarker_wide           DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cancer_10                DataFrame                PatientID Dia<...>n[11885 rows x 3 columns]
cancer_9                 DataFrame                PatientID Dia<...>\n[3718 rows x 3 columns]
cohort                   DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics             DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
diagnosis                DataFrame                PatientID    <...>[309101 rows x 9 columns]
diagnosis_elix_10        DataFrame                PatientID Dia<...>[63437 rows x 35 columns]
diagnosis_elix_10_wide   DataFrame              PatientID  chf <...>n[5199 rows x 33 columns]
diagnosis_elix_9         DataFrame                PatientID Dia<...>[17925 rows x 35 columns]
diagnosis_elix_9_wide    DataFrame              PatientID 

In [620]:
del cancer_10
del cancer_9
del diagnosis
del diagnosis_elix_10
del diagnosis_elix_10_wide
del diagnosis_elix_9
del diagnosis_elix_9_wide
del diagnosis_elixhauser
del other_cancer
del other_cancer_10
del other_cancer_9

10. Clean Social Determinants of Health Dataset

In [622]:
sdoh = pd.read_csv('../data/SocialDeterminantsOfHealth.csv')

In [623]:
sdoh = sdoh[sdoh['PatientID'].isin(cohort_IDs)]

In [624]:
row_ID(sdoh)

(6461, 6461)

In [625]:
sdoh.sample(3)

,PatientID,SESIndex2015_2019
1873,FD7667E941515,3
5201,F8F5A5015746B,5 - Highest SES
11696,F60AA4FB66DBF,2


In [626]:
sdoh['SESIndex2015_2019'].value_counts()

SESIndex2015_2019
4                  1396
3                  1250
5 - Highest SES    1152
2                  1118
1 - Lowest SES      876
Name: count, dtype: int64

In [627]:
conditions = [
    (sdoh['SESIndex2015_2019'] == '5 - Highest SES'),
    (sdoh['SESIndex2015_2019'] == '1 - Lowest SES')]    

choices = ['5', '1']
    
sdoh.loc[:, 'ses'] = np.select(conditions, choices, default = sdoh['SESIndex2015_2019'])

In [629]:
sdoh['ses'].value_counts()

ses
4    1396
3    1250
5    1152
2    1118
1     876
Name: count, dtype: int64

In [633]:
sdoh = sdoh.drop(columns = ['SESIndex2015_2019'])

KeyError: "['SESIndex2015_2019'] not found in axis"

In [634]:
row_ID(sdoh)

(6461, 6461)

In [631]:
# Append missing cohort IDs, unnecessary but to ensure in case of future data updates
sdoh_wide = (
    pd.concat(
        [sdoh, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(sdoh['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [632]:
row_ID(sdoh_wide)

(6461, 6461)

In [635]:
sdoh_wide.ses.value_counts(dropna = False, normalize = True)

ses
4      0.216066
3      0.193469
5      0.178301
2      0.173038
1      0.135583
NaN    0.103544
Name: proportion, dtype: float64

In [636]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
cohort                DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics          DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
diagnosis_wide        DataFrame              PatientID  chf <...>n[6461 rows x 36 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6461 rows x 2 columns]
enhanced_adv          DataFrame              PatientID diagn<...>n[6461 rows x 16 columns]
lab_wide              DataFrame              PatientID  albu<...>[6461 rows x 138 columns]
med_admin_wide        DataFrame              PatientID  opio<...>n[6461 rows x 14 columns]
sdoh                  DataFrame               PatientID  ses<...>\n[6461 rows x 2 columns]
sdoh_wide             DataFrame               PatientID  ses<...>\n[6461 rows x 2 columns]


In [637]:
del sdoh

11. Create mortality data file

In [754]:
mortality = pd.read_csv('../data/Enhanced_Mortality_V2.csv')

In [755]:
mortality = mortality[mortality['PatientID'].isin(cohort_IDs)]

In [756]:
row_ID(mortality)

(4501, 4501)

In [757]:
mortality = mortality.rename(columns = {'DateOfDeath': 'death_date'})

In [758]:
# For patients with year granularity, impute middle of the year (ie., July 1)
mortality['death_date'] = (
    np.where(mortality['death_date'].str.len() == 4, mortality['death_date'] + '-07-01', mortality['death_date'])
)

In [759]:
# For patients with month granularity, impute 15th of the month.
mortality['death_date'] = (
    np.where(mortality['death_date'].str.len() == 7, mortality['death_date'] + '-15', mortality['death_date'])
)

In [760]:
mortality['death_date'] = pd.to_datetime(mortality['death_date'], format="%Y-%m-%d")

Censoring
For patients for whom a date of death is not known, the censor date can be defined either as the data cutoff date or as the last confirmed activity date. The last confirmed activity date is broadly defined as the last date at which there is evidence in the EHR that a patient is alive. Evidence of a record in at least one of the items listed below qualifies as patient-level confirmed activity:

Visit: VisitDate
Telemedicine: VisitDate
Enhanced_AdvUrothelial_Orals: StartDate or EndDate
Enhanced_AdvUrothelialBiomarkers: SpecimenCollectedDate
Enhanced_AdvUrothelialProgression: LastClinicNoteDate or ProgressionDate

In [652]:
visit = pd.read_csv('../data/Visit.csv')
telemedicine = pd.read_csv('../data/Telemedicine.csv')
orals = pd.read_csv('../data/Enhanced_AdvUrothelial_Orals.csv')
biomarkers = pd.read_csv('../data/Enhanced_AdvUrothelialBiomarkers.csv')
progression = pd.read_csv('../data/Enhanced_AdvUrothelial_Progression.csv')

Visit and Telemedicine:

In [653]:
visit.shape

(574474, 7)

In [654]:
telemedicine.shape

(7380, 3)

In [656]:
visit_tele = pd.concat([visit[['PatientID', 'VisitDate']], telemedicine[['PatientID', 'VisitDate']]], ignore_index=True)

In [657]:
visit_tele.shape

(581854, 2)

In [658]:
visit_tele['VisitDate'] = pd.to_datetime(visit_tele['VisitDate'], format="%Y-%m-%d")

In [660]:
# Select max VisitDate from combined Visit and Telemedicine table.
visit_tele_max = (
    visit_tele
    [visit_tele['PatientID'].isin(cohort_IDs)]
    .groupby('PatientID')['VisitDate'].max()
    .to_frame(name = 'visit_max')
    .reset_index()
)

In [661]:
row_ID(visit_tele_max)

(6461, 6461)

Orals

In [662]:
orals = orals[orals['PatientID'].isin(cohort_IDs)]

In [663]:
orals.loc[:, 'StartDate'] = pd.to_datetime(orals['StartDate'], format="%Y-%m-%d")

In [664]:
orals.loc[:, 'EndDate'] = pd.to_datetime(orals['EndDate'], format="%Y-%m-%d")

In [665]:
orals_max = (
    orals
    .assign(max_date = orals[['StartDate', 'EndDate']].max(axis = 1))
    .groupby('PatientID')['max_date'].max()
    .to_frame(name = 'orals_max')
    .reset_index()
)

Biomarkers

In [666]:
biomarkers = biomarkers[biomarkers['PatientID'].isin(cohort_IDs)]

In [667]:
biomarkers.loc[:, 'SpecimenCollectedDate'] = pd.to_datetime(biomarkers['SpecimenCollectedDate'], format="%Y-%m-%d")

In [668]:
biomarkers_max = (
    biomarkers
    .groupby('PatientID')['SpecimenCollectedDate'].max()
    .to_frame(name = 'biomarkers_max')
    .reset_index()
)

Progression

In [669]:
progression = progression[progression['PatientID'].isin(cohort_IDs)]

In [675]:
progression.loc[:, 'ProgressionDate'] = pd.to_datetime(progression['ProgressionDate'], format="%Y-%m-%d", errors = 'coerce')

In [677]:
progression.loc[:, 'LastClinicNoteDate'] = pd.to_datetime(progression['LastClinicNoteDate'], format="%Y-%m-%d", errors = 'coerce')

In [680]:
print(progression[['ProgressionDate', 'LastClinicNoteDate']].dtypes)
print(progression['ProgressionDate'].unique())
print(progression['LastClinicNoteDate'].unique())
print(progression['ProgressionDate'].isna().sum())  # Count NaT in ProgressionDate
print(progression['LastClinicNoteDate'].isna().sum())  # Count NaT in LastClinicNoteDate

ProgressionDate       object
LastClinicNoteDate    object
dtype: object
[Timestamp('2022-11-29 00:00:00') Timestamp('2023-01-27 00:00:00') NaT ...
 Timestamp('2011-11-09 00:00:00') Timestamp('2022-06-08 00:00:00')
 Timestamp('2023-08-14 00:00:00')]
[Timestamp('2023-08-31 00:00:00') Timestamp('2014-01-28 00:00:00')
 Timestamp('2023-06-27 00:00:00') ... Timestamp('2019-05-19 00:00:00')
 Timestamp('2012-01-11 00:00:00') Timestamp('2021-07-16 00:00:00')]
2356
0


Need to address missing values in ProgressionDate. There are no missing LastClinicNoteDate values, so we can conver the ProgressionDate values that are missing to an early timestamp so that the max of the two will be the LastClinicNoteDate

In [682]:
progression['ProgressionDate'] = progression['ProgressionDate'].fillna(pd.Timestamp('1900-01-01'))

In [683]:
print(progression['ProgressionDate'].isna().sum())  # Count NaT in ProgressionDate
print(progression['LastClinicNoteDate'].isna().sum())  # Count NaT in LastClinicNoteDate

0
0


In [684]:
progression_max = (
    progression
    .assign(max_date = progression[['ProgressionDate', 'LastClinicNoteDate']].max(axis = 1, skipna=True))
    .groupby('PatientID')['max_date'].max()
    .to_frame(name = 'progression_max')
    .reset_index()
)

Max date merge

In [685]:
last_activity = pd.merge(visit_tele_max, orals_max, on = 'PatientID', how = 'outer')

In [686]:
last_activity = pd.merge(last_activity, biomarkers_max, on = 'PatientID', how = 'outer')

In [687]:
last_activity = pd.merge(last_activity, progression_max, on = 'PatientID', how = 'outer')

In [688]:
row_ID(last_activity)

(6461, 6461)

In [690]:
last_activity = (
    last_activity
    .assign(last_activity = last_activity[['visit_max', 'orals_max', 'biomarkers_max', 'progression_max']].max(axis = 1))
    .filter(items = ['PatientID', 'last_activity'])
)

TypeError: '>=' not supported between instances of 'Timestamp' and 'float'

In [692]:
print(last_activity['visit_max'].isna().sum())

0


In [693]:
print(last_activity['orals_max'].isna().sum())

6321


In [694]:
print(last_activity['biomarkers_max'].isna().sum())

3913


In [695]:
print(last_activity['progression_max'].isna().sum())

5


There are cases of missing values for orals, biomarkers, progression. As a result, we will fill.na with an old date to make sure that visti_max is the default.

In [698]:
last_activity['orals_max'] = last_activity['orals_max'].fillna(pd.Timestamp('1900-01-01'))
last_activity['biomarkers_max'] = last_activity['biomarkers_max'].fillna(pd.Timestamp('1900-01-01'))
last_activity['progression_max'] = last_activity['progression_max'].fillna(pd.Timestamp('1900-01-01'))

/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/368745325.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  last_activity['biomarkers_max'] = last_activity['biomarkers_max'].fillna(pd.Timestamp('1900-01-01'))
/var/folders/q7/3mfjt48n48v4fyx7k0bgxfk40000gn/T/ipykernel_2878/368745325.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  last_activity['progression_max'] = last_activity['progression_max'].fillna(pd.Timestamp('1900-01-01'))


In [699]:
last_activity = (
    last_activity
    .assign(last_activity = last_activity[['visit_max', 'orals_max', 'biomarkers_max', 'progression_max']].max(axis = 1))
    .filter(items = ['PatientID', 'last_activity'])
)

In [700]:
last_activity['last_activity'].isna().sum()

np.int64(0)

In [701]:
row_ID(mortality)

(4501, 4501)

In [761]:
# Append missing cohort IDs
mortality = (
    pd.concat(
        [mortality, 
        pd.Series(cohort_IDs)[~pd.Series(cohort_IDs).isin(mortality['PatientID'])].to_frame(name = 'PatientID')],
        sort = False)
)

In [762]:
mortality = pd.merge(mortality, cohort[['PatientID', 'StartDate']], on = 'PatientID')
mortality['StartDate'] = pd.to_datetime(mortality['StartDate'], format="%Y-%m-%d")

In [763]:
mortality = pd.merge(mortality, last_activity, on = 'PatientID')

In [764]:
print(mortality.dtypes)


PatientID                object
death_date       datetime64[ns]
StartDate        datetime64[ns]
last_activity    datetime64[ns]
dtype: object


In [765]:
mortality.loc[:, 'death_status'] = np.where(mortality['death_date'].isna(), 0, 1)

In [766]:
print(mortality.dtypes)

PatientID                object
death_date       datetime64[ns]
StartDate        datetime64[ns]
last_activity    datetime64[ns]
death_status              int64
dtype: object


In [767]:
# timerisk_activity is time from metastatic diagnosis to death or last activity if no death date.
mortality.loc[:, 'timerisk_activity'] = (
    np.where(mortality['death_date'].isna(),
             (mortality['last_activity'] - mortality['StartDate']).dt.days,
             (mortality['death_date'] - mortality['StartDate']).dt.days)
)

In [768]:
# If timerisk_activity is less than 0, set to 0 otherwise remains unchanged. 
mortality['timerisk_activity'] = np.where(mortality['timerisk_activity'] < 0, 0, mortality['timerisk_activity'])

In [769]:
mortality.sample(3)

,PatientID,death_date,StartDate,last_activity,death_status,timerisk_activity
5255,F7DEDFBB01ECA,NaT,2022-05-03,2023-10-24,0,539.0
540,F91B7360D6194,2014-09-15,2013-08-13,2014-08-15,1,398.0
5164,FAB1D8257FF31,NaT,2023-08-21,2023-10-09,0,49.0


In [770]:
row_ID(mortality)

(6461, 6461)

In [771]:
mortality = mortality.filter(items = ['PatientID', 'death_status', 'timerisk_activity'])

In [772]:
mortality.sample(3)

,PatientID,death_status,timerisk_activity
1848,FC689F3653E3E,1,774.0
6239,F4E890892DB78,0,3632.0
4398,F20EB67348A06,1,138.0


In [773]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
biomarker_wide        DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
biomarkers            DataFrame              PatientID Bioma<...>n[6326 rows x 19 columns]
biomarkers_max        DataFrame              PatientID      <...>\n[2623 rows x 2 columns]
cohort                DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics          DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
diagnosis_wide        DataFrame              PatientID  chf <...>n[6461 rows x 36 columns]
ecog_diagnosis_wide   DataFrame                PatientID eco<...>\n[6461 rows x 2 columns]
enhanced_adv          DataFrame              PatientID prima<...>n[6461 rows x 14 columns]
failed_conversion     DataFrame    Empty DataFrame\nColumns:<...> death_status]\nIndex: []
lab_wide              DataFrame              PatientID  albu<...>[6461 rows x 138 columns]


In [774]:
del biomarkers
del biomarkers_max
del last_activity
del orals
del orals_max
del telemedicine
del visit
del visit_tele
del visit_tele_max

12. Merge all files together to create final dataframe

In [775]:
test_full = pd.merge(demographics, enhanced_adv, on = 'PatientID')

In [776]:
test_full = pd.merge(test_full, mortality, on = 'PatientID')

In [777]:
test_full = pd.merge(test_full, med_admin_wide, on = 'PatientID')

In [778]:
test_full = pd.merge(test_full, biomarker_wide, on = 'PatientID')

In [779]:
test_full = pd.merge(test_full, ecog_diagnosis_wide, on = 'PatientID')

In [780]:
test_full = pd.merge(test_full, weight_wide, on = 'PatientID')

In [781]:
test_full = pd.merge(test_full, vitals_wide, on = 'PatientID')

In [782]:
test_full = pd.merge(test_full, lab_wide, on = 'PatientID')

In [783]:
test_full = pd.merge(test_full, diagnosis_wide, on = 'PatientID')

In [784]:
test_full = pd.merge(test_full, sdoh_wide, on = 'PatientID')

In [785]:
row_ID(test_full)

(6461, 6461)

In [787]:
test_full.shape

(6461, 221)

In [788]:
list(test_full.columns)

['PatientID',
 'gender',
 'race',
 'ethnicity',
 'age',
 'practice_type',
 'region',
 'primary_site',
 'disease_grade',
 't_stage',
 'n_stage',
 'm_stage',
 'smoking_status',
 'surgery_status',
 'surgery_type',
 'stage',
 'adv_diagnosis_year',
 'delta_adv_diagnosis',
 'delta_therapy_surgery',
 'delta_therapy_diagnosis',
 'death_status',
 'timerisk_activity',
 'opioid_PO_diag',
 'nonopioid_PO_diag',
 'pain_IV_diag',
 'ac_diag',
 'antiinfective_IV_diag',
 'antiinfective_diag',
 'steroid_diag',
 'antihyperglycemic_diag',
 'ppi_diag',
 'antidepressant_diag',
 'thyroid_diag',
 'bta_diag',
 'is_diag',
 'FGFR',
 'pdl1_n',
 'ecog_diagnosis',
 'weight_diag',
 'bmi_diag',
 'weight_pct_change',
 'weight_pct_na',
 'weight_slope',
 'sbp_diag',
 'dbp_diag',
 'hr_diag',
 'pulse_ox_diag',
 'temp_diag',
 'albumin_diag',
 'alp_diag',
 'alt_diag',
 'ast_diag',
 'bicarb_diag',
 'bun_diag',
 'calcium_diag',
 'chloride_diag',
 'creatinine_diag',
 'hemoglobin_diag',
 'lymphocyte_count_diag',
 'neutrophil_cou

In [789]:
test_full.to_csv('test_full.csv', index = False, header = True)

In [790]:
test_full.sample(3)

,PatientID,gender,race,ethnicity,age,practice_type,region,primary_site,disease_grade,t_stage,...,deficiency_anemia,alcohol_abuse,drug_abuse,psychoses,depression,elixhauser_other,icd_count,elixhauser_index,other_cancer,ses
989,FB40E8A6F6F39,M,White,not_hispanic_latino,60,ACADEMIC,unknown,lower_tract,high_grade,T2,...,0.0,0.0,0.0,0.0,0.0,1.0,55.0,5.0,0.0,5
5347,F671913EE1ABD,M,White,not_hispanic_latino,69,COMMUNITY,northeast,lower_tract,high_grade,T1,...,0.0,0.0,0.0,0.0,0.0,1.0,11.0,6.0,0.0,4
948,F7BD791763393,M,White,not_hispanic_latino,72,ACADEMIC,unknown,upper_tract,high_grade,unknown,...,0.0,0.0,0.0,0.0,1.0,1.0,136.0,9.0,1.0,4
